# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Network-Learning-Parameter" data-toc-modified-id="Network-Learning-Parameter-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Network Learning Parameter</a></div><div class="lev1 toc-item"><a href="#Data" data-toc-modified-id="Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data</a></div><div class="lev1 toc-item"><a href="#ATTRNN" data-toc-modified-id="ATTRNN-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>ATTRNN</a></div><div class="lev4 toc-item"><a href="#Run" data-toc-modified-id="Run-3001"><span class="toc-item-num">3.0.0.1&nbsp;&nbsp;</span>Run</a></div><div class="lev3 toc-item"><a href="#ATTRNN" data-toc-modified-id="ATTRNN-301"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>ATTRNN</a></div><div class="lev4 toc-item"><a href="#Run" data-toc-modified-id="Run-3011"><span class="toc-item-num">3.0.1.1&nbsp;&nbsp;</span>Run</a></div><div class="lev1 toc-item"><a href="#RNNCell-Wrapper" data-toc-modified-id="RNNCell-Wrapper-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>RNNCell Wrapper</a></div><div class="lev3 toc-item"><a href="#Training" data-toc-modified-id="Training-401"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>Training</a></div><div class="lev2 toc-item"><a href="#RNN-Wrapper" data-toc-modified-id="RNN-Wrapper-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>RNN Wrapper</a></div><div class="lev3 toc-item"><a href="#fullrnn" data-toc-modified-id="fullrnn-411"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>fullrnn</a></div><div class="lev3 toc-item"><a href="#RNN-Wrapper" data-toc-modified-id="RNN-Wrapper-412"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>RNN Wrapper</a></div><div class="lev4 toc-item"><a href="#Training" data-toc-modified-id="Training-4121"><span class="toc-item-num">4.1.2.1&nbsp;&nbsp;</span>Training</a></div><div class="lev3 toc-item"><a href="#ATTRNN" data-toc-modified-id="ATTRNN-413"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>ATTRNN</a></div><div class="lev4 toc-item"><a href="#Training" data-toc-modified-id="Training-4131"><span class="toc-item-num">4.1.3.1&nbsp;&nbsp;</span>Training</a></div><div class="lev3 toc-item"><a href="#ATTRNN-Wrapper" data-toc-modified-id="ATTRNN-Wrapper-414"><span class="toc-item-num">4.1.4&nbsp;&nbsp;</span>ATTRNN Wrapper</a></div><div class="lev4 toc-item"><a href="#Training" data-toc-modified-id="Training-4141"><span class="toc-item-num">4.1.4.1&nbsp;&nbsp;</span>Training</a></div><div class="lev1 toc-item"><a href="#GRU-Decoder-with-Attention-(encoder:-return_sequence=True)" data-toc-modified-id="GRU-Decoder-with-Attention-(encoder:-return_sequence=True)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>GRU Decoder with Attention (encoder: <code>return_sequence=True</code>)</a></div><div class="lev3 toc-item"><a href="#MyGRUAttention-(Feed-Forward,-Not-Recurrent)" data-toc-modified-id="MyGRUAttention-(Feed-Forward,-Not-Recurrent)-501"><span class="toc-item-num">5.0.1&nbsp;&nbsp;</span>MyGRUAttention (Feed-Forward, Not Recurrent)</a></div><div class="lev3 toc-item"><a href="#Modeling" data-toc-modified-id="Modeling-502"><span class="toc-item-num">5.0.2&nbsp;&nbsp;</span>Modeling</a></div><div class="lev3 toc-item"><a href="#Training" data-toc-modified-id="Training-503"><span class="toc-item-num">5.0.3&nbsp;&nbsp;</span>Training</a></div><div class="lev3 toc-item"><a href="#Scoring" data-toc-modified-id="Scoring-504"><span class="toc-item-num">5.0.4&nbsp;&nbsp;</span>Scoring</a></div><div class="lev3 toc-item"><a href="#Testing" data-toc-modified-id="Testing-505"><span class="toc-item-num">5.0.5&nbsp;&nbsp;</span>Testing</a></div>

In [3]:
import os
import math
import datetime as dt
import itertools as it
from glob import glob
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import (SimpleRNN, RNN, LSTM, GRU,
                          Input, Reshape, Dense, Flatten, Permute, Lambda,
                          Embedding, RepeatVector, Activation,
                          TimeDistributed, Bidirectional,
                          dot, multiply, concatenate, merge)
from keras.callbacks import Callback, LambdaCallback
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.utils import multi_gpu_model

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.
/home/pydemia/apps/anaconda3/envs/tf-py36/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [40]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.weights = []
        self.states = []

    def on_batch_begin(self, batch, logs={}):
        self.weights.append([{'begin_' + layer.name: layer.get_weights()} for layer in model.layers])

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.weights.append([{'end_' + layer.name: layer.get_weights()} for layer in model.layers])
        

history = LossHistory()

print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: pprint(model.layers[0].get_weights()))
#print_outputs = LambdaCallback(on_epoch_end=lambda batch, logs: pprint(model.layers[2].output))
#print_states = LambdaCallback(on_epoch_end=lambda batch, logs: pprint(model.layers[2].states))

# Network Learning Parameter

In [5]:
LATENT_DIM = HIDDEN_SIZE = 64
EPOCH_NUM = 100
BATCH_SIZE = 256
#GPU_NUM = 1

# Data

Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model.  
We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

* Summary of the algorithm
    - We start with input sequences from a domain (e.g. English sentences)
        and correspding target sequences from another domain
        (e.g. French sentences).
    - An encoder LSTM turns input sequences to 2 state vectors
        (we keep the last LSTM state and discard the outputs).
    - A decoder LSTM is trained to turn the target sequences into
        the same sequence but offset by one timestep in the future,
        a training process called "teacher forcing" in this context.  
        Is uses as initial state the state vectors from the encoder.
        Effectively, the decoder learns to generate `targets[t+1...]`
        given `targets[...t]`, conditioned on the input sequence.
    - In inference mode, when we want to decode unknown input sequences, we:
        - Encode the input sequence into state vectors
        - Start with a target sequence of size 1
            (just the start-of-sequence character)
        - Feed the state vectors and 1-char target sequence
            to the decoder to produce predictions for the next character
        - Sample the next character using these predictions
            (we simply use argmax).
        - Append the sampled character to the target sequence
        - Repeat until we generate the end-of-sequence character or we
            hit the character limit.
* Data download
    English to French sentence pairs.
    http://www.manythings.org/anki/fra-eng.zip
    Lots of neat sentence pairs datasets can be found at:
    http://www.manythings.org/anki/

* References
    - Sequence to Sequence Learning with Neural Networks
        https://arxiv.org/abs/1409.3215
    - Learning Phrase Representations using
        RNN Encoder-Decoder for Statistical Machine Translation
        https://arxiv.org/abs/1406.1078


In [6]:
import numpy as np

train_X = np.random.sample(300).reshape(-1, 3, 2)
train_Y = np.random.sample(300).reshape(-1, 3, 2)

In [7]:
train_X[:2]

array([[[ 0.48190335,  0.37729399],
        [ 0.17947748,  0.50321185],
        [ 0.1484575 ,  0.80850603]],

       [[ 0.2486874 ,  0.18620542],
        [ 0.1939146 ,  0.40464986],
        [ 0.52080591,  0.13876185]]])

In [8]:
train_Y[:2]

array([[[ 0.20182727,  0.66169752],
        [ 0.82607336,  0.71335799],
        [ 0.74202261,  0.13219845]],

       [[ 0.26560703,  0.1766803 ],
        [ 0.97170505,  0.43612362],
        [ 0.41986697,  0.10127776]]])

!['lstm_many_to_many_1'](lstm_many_to_many_1.jpg)

!['lstm_attention'](rnn_with_att.jpg)
!['Overview of the Attention mechanism in an Encoder-Decoder setup'](lstm_attention_3.png)
!['detail_lstm_attention'](detail_attentionmodel1.png)

[Attention Structure 1](https://blog.heuritech.com/2016/01/20/attention-mechanism/)  
[Attention Structure 2](http://www.wildml.com/2016/01/attention-and-memory-in-deep-learning-and-nlp/)  
[Attention Structure 3](https://medium.com/datalogue/attention-in-keras-1892773a4f22)  

`keras.layers.Embedding`:  
> `(nb_words, vocab_size) x (vocab_size, embedding_dim) = (nb_words, embedding_dim)`

---
# ATTRNN

In [10]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
import numpy as np
import warnings

from keras import backend as K
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.engine import Layer
from keras.engine import InputSpec
from keras.engine.topology import _object_list_uid
from keras.utils.generic_utils import has_arg

# Legacy support.
from keras.legacy.layers import Recurrent
from keras.legacy import interfaces

from keras.layers import Input
from keras.layers import (RNN,
                          SimpleRNNCell,
                          LSTMCell, LSTM,
                          GRUCell, GRU,
                          Layer, Input,
                          Wrapper)

from keras.layers.recurrent import *
from keras import Model
#from keras.layers.recurrent import (_generate_dropout_ones,
#                                    _generate_dropout_mask)

# %% ATTRNNCell ---------------------------------------------------------------

class ATTRNNCell(Layer):

    def __init__(self, units, attn_size, attn_length,
                 activation='tanh',
                 **kwargs):
        super(ATTRNNCell, self).__init__(**kwargs)
        self.units = units
        self._attn_size = attn_size
        self._attn_length = attn_length
        self._state_size = self.units
        #self.state_size = (self.units,
        #                   self._attn_size,
        #                   self._attn_size * self._attn_length)
        self.activation = activations.get(activation)

    @property
    def state_size(self):
        #state_size = self.units

        state_size_a = []
        for state_size in (self.units, self.units):
            state_size_a.append(state_size)

        return state_size_a

    #@state_size.setter
    #def state_size(self, state_size):
    #    self.state_size = state_size

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units),
                                      initializer='uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='uniform',
            name='recurrent_kernel')
        self.built = True

        self.attn_kernel = self.add_weight(shape=(input_dim, self.units),
                                           initializer='uniform',
                                           name='attn_kernel')

    def call(self, inputs, states, constants=None):
        prev_output = states[0]
        attn_state = states[1:]
        h = K.dot(inputs, self.kernel)
        output = h + K.dot(prev_output, self.recurrent_kernel)
        output = self.activation(output)
        return output, [output] + list(attn_state)


#### Run

In [13]:
aa = Input(shape=(3, 1), dtype='float32')
bb = ATTRNNCell(2, 3, 3)
cc = RNN(bb, return_sequences=True, return_state=True)(aa)
#cc = LSTM(2, return_sequences=True, return_state=True)(aa)
dd = Model(inputs=aa, outputs=cc)
dd.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 1)              0         
_________________________________________________________________
rnn_1 (RNN)                  [(None, 3, 2), (None, 2), 8         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


### ATTRNN

In [15]:

# %% ATTRNN Layer -------------------------------------------------------------

class ATTRNN(RNN):

    def __init__(self, cell,
                 return_sequences=False,
                 return_state=False,
                 go_backwards=False,
                 stateful=False,
                 unroll=False,
                 **kwargs):
        super(ATTRNN, self).__init__(cell, **kwargs)
        #self.cell = cell
        #self.return_sequences = return_sequences
        #self.return_state = return_state
        #self.go_backwards = go_backwards
        #self.stateful = stateful
        #self.unroll = unroll

        #self.supports_masking = True
        #self.input_spec = [InputSpec(ndim=3)]
        #self.state_spec = None
        #self._states = None
        #self.constants_spec = None
        #self._num_constants = None

    def build(self, input_shape):
        # Note input_shape will be list of shapes of initial states and
        # constants if these are passed in __call__.
        if self._num_constants is not None:
            constants_shape = input_shape[-self._num_constants:]
        else:
            constants_shape = None

        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        batch_size = input_shape[0] if self.stateful else None
        input_dim = input_shape[-1]
        self.input_spec[0] = InputSpec(shape=(batch_size, None, input_dim))

        # allow cell (if layer) to build before we set or validate state_spec
        if isinstance(self.cell, Layer):
            step_input_shape = (input_shape[0],) + input_shape[2:]
            if constants_shape is not None:
                self.cell.build([step_input_shape] + constants_shape)
            else:
                self.cell.build(step_input_shape)

        # set or validate state_spec
        if hasattr(self.cell.state_size, '__len__'):
            state_size = list(self.cell.state_size)
        else:
            state_size = [self.cell.state_size]

        if self.state_spec is not None:
            # initial_state was passed in call, check compatibility
            if not [spec.shape[-1] for spec in self.state_spec] == state_size:
                raise ValueError(
                    'An initial_state was passed that is not compatible with '
                    '`cell.state_size`. Received `state_spec`={}; '
                    'However `cell.state_size` is '
                    '{}'.format(self.state_spec, self.cell.state_size))
        else:
            self.state_spec = [InputSpec(shape=(None, dim))
                               for dim in state_size]
        if self.stateful:
            self.reset_states()

    def get_initial_state(self, inputs):
        return super(ATTRNN, self).get_initial_state(inputs)

    def __call__(self, inputs, initial_state=None, constants=None, **kwargs):
        inputs, initial_state, constants = self._standardize_args(
            inputs, initial_state, constants)

        if initial_state is None and constants is None:
            return super(RNN, self).__call__(inputs, **kwargs)

        # If any of `initial_state` or `constants` are specified and are Keras
        # tensors, then add them to the inputs and temporarily modify the
        # input_spec to include them.

        additional_inputs = []
        additional_specs = []
        if initial_state is not None:
            kwargs['initial_state'] = initial_state
            additional_inputs += initial_state
            self.state_spec = [InputSpec(shape=K.int_shape(state))
                               for state in initial_state]
            additional_specs += self.state_spec
        if constants is not None:
            kwargs['constants'] = constants
            additional_inputs += constants
            self.constants_spec = [InputSpec(shape=K.int_shape(constant))
                                   for constant in constants]
            self._num_constants = len(constants)
            additional_specs += self.constants_spec
        # at this point additional_inputs cannot be empty
        is_keras_tensor = hasattr(additional_inputs[0], '_keras_history')
        for tensor in additional_inputs:
            if hasattr(tensor, '_keras_history') != is_keras_tensor:
                raise ValueError('The initial state or constants of an RNN'
                                 ' layer cannot be specified with a mix of'
                                 ' Keras tensors and non-Keras tensors')

        if is_keras_tensor:
            # Compute the full input spec, including state and constants
            full_input = [inputs] + additional_inputs
            full_input_spec = self.input_spec + additional_specs
            # Perform the call with temporarily replaced input_spec
            original_input_spec = self.input_spec
            self.input_spec = full_input_spec
            output = super(RNN, self).__call__(full_input, **kwargs)
            self.input_spec = original_input_spec
            return output
        else:
            return super(RNN, self).__call__(inputs, **kwargs)

    def call(self,
             inputs,
             mask=None,
             training=None,
             initial_state=None,
             constants=None):
        # input shape: `(samples, time (padded with zeros), input_dim)`
        # note that the .build() method of subclasses MUST define
        # self.input_spec and self.state_spec with complete input shapes.
        if isinstance(inputs, list):
            inputs = inputs[0]
        if initial_state is not None:
            pass
        elif self.stateful:
            initial_state = self.states
        else:
            initial_state = self.get_initial_state(inputs)

        if isinstance(mask, list):
            mask = mask[0]

        if len(initial_state) != len(self.states):
            raise ValueError('Layer has ' + str(len(self.states)) +
                             ' states but was passed ' +
                             str(len(initial_state)) +
                             ' initial states.')
        input_shape = K.int_shape(inputs)
        timesteps = input_shape[1]
        if self.unroll and timesteps in [None, 1]:
            raise ValueError('Cannot unroll a RNN if the '
                             'time dimension is undefined or equal to 1. \n'
                             '- If using a Sequential model, '
                             'specify the time dimension by passing '
                             'an `input_shape` or `batch_input_shape` '
                             'argument to your first layer. If your '
                             'first layer is an Embedding, you can '
                             'also use the `input_length` argument.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a `shape` '
                             'or `batch_shape` argument to your Input layer.')

        kwargs = {}
        if has_arg(self.cell.call, 'training'):
            kwargs['training'] = training

        if constants:
            if not has_arg(self.cell.call, 'constants'):
                raise ValueError('RNN cell does not support constants')

            def step(inputs, states):
                constants = states[-self._num_constants:]
                states = states[:-self._num_constants]
                return self.cell.call(inputs, states, constants=constants,
                                      **kwargs)
        else:
            def step(inputs, states):
                return self.cell.call(inputs, states, **kwargs)

        last_output, outputs, states = fullrnn(step,
                                             inputs,
                                             initial_state,
                                             constants=constants,
                                             go_backwards=self.go_backwards,
                                             mask=mask,
                                             unroll=self.unroll,
                                             input_length=timesteps)
        if self.stateful:
            updates = []
            for i in range(len(states)):
                updates.append((self.states[i], states[i]))
            self.add_update(updates, inputs)

        if self.return_sequences:
            output = outputs
        else:
            output = last_output

        # Properly set learning phase
        if getattr(last_output, '_uses_learning_phase', False):
            output._uses_learning_phase = True

        if self.return_state:
            if not isinstance(states, (list, tuple)):
                states = [states]
            else:
                states = list(states)
            return [output] + states
        else:
            return output
    
    def get_attention(self, inputs):
        pass

#### Run

In [18]:
# %% RUN ----------------------------------------------------------------------

aa = Input(shape=(3, 1), dtype='float32')
cc = ATTRNN(ATTRNNCell(2, 3, 3), return_sequences=True, return_state=True)(aa)
#cc = LSTM(2, return_sequences=True, return_state=True)(aa)
dd = Model(inputs=aa, outputs=cc)
dd.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 3, 1)              0         
_________________________________________________________________
attrnn_2 (ATTRNN)            (None, 2)                 8         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


In [20]:
# %% RUN ----------------------------------------------------------------------

aa = Input(shape=(3, 1), dtype='float32')
bb = Input(tensor=aa)
#cc = RNN(ATTRNNCell(2, 3, 3), return_sequences=True, return_state=True)(aa, constants=bb)
cc = ATTRNN(LSTMCell(2), return_sequences=True, return_state=True)(aa)
dd = Model(inputs=aa, outputs=cc)
dd.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 3, 1)              0         
_________________________________________________________________
attrnn_3 (ATTRNN)            (None, 2)                 32        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________


---
# RNNCell Wrapper

In [23]:
class CellWrapper(Layer):

    """
    A Cell Wrapper for Attention Mechanism.
    """

    def __init__(self, cell, *args, **kwargs):
        super(CellWrapper, self).__init__(*args, **kwargs)
        self._cell = cell
        self._cell.__init__(self.units, **kwargs)

        self.trainable = self._cell.trainable

    @property
    def units(self):
        return self._cell.units

    @property
    def activation(self):
        return self._cell.activation

    @property
    def use_bias(self):
        return self._cell.use_bias

    @property
    def kernel_initializer(self):
        return self._cell.kernel_initializer

    @property
    def recurrent_initializer(self):
        return self._cell.recurrent_initializer

    @property
    def bias_initializer(self):
        return self._cell.bias_initializer

    @property
    def kernel_regularizer(self):
        return self._cell.kernel_regularizer

    @property
    def recurrent_regularizer(self):
        return self._cell.recurrent_regularizer

    @property
    def bias_regularizer(self):
        return self._cell.bias_regularizer

    @property
    def kernel_constraint(self):
        return self._cell.kernel_constraint

    @property
    def recurrent_constraint(self):
        return self._cell.recurrent_constraint

    @property
    def bias_constraint(self):
        return self._cell.bias_constraint

    @property
    def dropout(self):
        return self._cell.dropout

    @property
    def recurrent_dropout(self):
        return self._cell.recurrent_dropout

    @property
    def state_size(self):
        return self._cell.state_size

    @property
    def output_size(self):
        return self._cell.output_size

    @property
    def implementation(self):
        return self.cell.implementation

#    @property
#    def weights(self):
#        return self._cell.weights + super(CellWrapper, self).weights

#    @property
#    def get_weights(self):
#        #return self._cell.weights + self.weights
#        #return self.weights
#
#        params = self.weights
#        return K.batch_get_value(params)

    def build(self, input_shape):

        self._cell.build(input_shape)
        if self._cell.trainable:
            self._trainable_weights.extend(self._cell.weights)
        else:
            self._non_trainable_weights.extend(self._cell.weights)

        input_dim = input_shape[-1]
        self.kk = self.add_weight(shape=(input_dim, self.units,),
                                  name='wrapper_kernel',
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)

        self.bb = self.add_weight(shape=(self.units,),
                                  name='wrapper_bias',
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        """
        self.kernel
        self.recurrent_kernel

        """
        self.built = True

    def call(self, inputs, states, training=None):
        new_inputs, new_states = self._cell.call(inputs, states,
                                                 training=training)

        new_inputs = K.dot(self.kk, new_inputs)
        new_inputs = K.bias_add(new_inputs, self.bb)

        return new_inputs, new_states

In [24]:
aa = Input(shape=(3, 2), dtype='float32')
bb = SimpleRNNCell(2)
cc = CellWrapper(bb)
dd = RNN(cc, return_sequences=True, return_state=False)(aa)
ee = Model(inputs=aa, outputs=dd)
ee.compile(optimizer='adam', loss='mae', metrics=['mae'])
ee.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3, 2)              0         
_________________________________________________________________
rnn_3 (RNN)                  (None, 3, 2)              16        
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


In [25]:
cc.trainable_weights

[<tf.Variable 'rnn_3/kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/recurrent_kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/bias:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'rnn_3/wrapper_kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/wrapper_bias:0' shape=(2,) dtype=float32_ref>]

In [26]:
bb.weights

[<tf.Variable 'rnn_3/kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/recurrent_kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/bias:0' shape=(2,) dtype=float32_ref>]

In [27]:
bb.get_weights()

[array([[ 1.21733773,  0.08077967],
        [-0.56632608, -1.07468975]], dtype=float32),
 array([[-0.99097782,  0.13402598],
        [ 0.13402598,  0.99097782]], dtype=float32),
 array([ 0.,  0.], dtype=float32)]

In [30]:
cc.weights

[<tf.Variable 'rnn_3/kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/recurrent_kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/bias:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'rnn_3/wrapper_kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/wrapper_bias:0' shape=(2,) dtype=float32_ref>]

In [31]:
cc.get_weights()

[array([[ 1.21733773,  0.08077967],
        [-0.56632608, -1.07468975]], dtype=float32),
 array([[-0.99097782,  0.13402598],
        [ 0.13402598,  0.99097782]], dtype=float32),
 array([ 0.,  0.], dtype=float32),
 array([[ 0.89775884, -0.07153285],
        [-0.35253835, -0.80943173]], dtype=float32),
 array([ 0.14992344,  1.28210735], dtype=float32)]

In [32]:
cc.trainable

True

In [33]:
cc.trainable_weights

[<tf.Variable 'rnn_3/kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/recurrent_kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/bias:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'rnn_3/wrapper_kernel:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'rnn_3/wrapper_bias:0' shape=(2,) dtype=float32_ref>]

In [34]:
cc.count_params()

16

### Training

In [41]:
print('EPOCH_NUM: %s, BATCH_SIZE %s' % (EPOCH_NUM, BATCH_SIZE))

model = ee
fitted = model.fit(train_X, train_Y,
                   epochs=10,     # How many times to run back_propagation
                   batch_size=2,  # How many data to deal with at one epoch
                   validation_split=0.2,
                   verbose=2,       # 1: progress bar, 2: one line per epoch
                   #validation_data=(testX, testY),  # Validation set
                   shuffle=True,
                   callbacks=[history],
                  )

# Save model
model.save('gru_attention_embedding_model.h5')

EPOCH_NUM: 100, BATCH_SIZE 256
Train on 40 samples, validate on 10 samples
Epoch 1/10
 - 0s - loss: 0.4883 - mean_absolute_error: 0.4883 - val_loss: 0.4634 - val_mean_absolute_error: 0.4634
Epoch 2/10
 - 0s - loss: 0.4962 - mean_absolute_error: 0.4962 - val_loss: 0.4515 - val_mean_absolute_error: 0.4515
Epoch 3/10
 - 0s - loss: 0.4707 - mean_absolute_error: 0.4707 - val_loss: 0.4395 - val_mean_absolute_error: 0.4395
Epoch 4/10
 - 0s - loss: 0.4717 - mean_absolute_error: 0.4717 - val_loss: 0.4276 - val_mean_absolute_error: 0.4276
Epoch 5/10
 - 0s - loss: 0.4313 - mean_absolute_error: 0.4313 - val_loss: 0.4182 - val_mean_absolute_error: 0.4182
Epoch 6/10
 - 0s - loss: 0.3984 - mean_absolute_error: 0.3984 - val_loss: 0.4073 - val_mean_absolute_error: 0.4073
Epoch 7/10
 - 0s - loss: 0.4118 - mean_absolute_error: 0.4118 - val_loss: 0.3973 - val_mean_absolute_error: 0.3973
Epoch 8/10
 - 0s - loss: 0.4204 - mean_absolute_error: 0.4204 - val_loss: 0.3904 - val_mean_absolute_error: 0.3904
Epoch

In [42]:
pprint(history.weights[:4])

[[{'begin_input_11': []},
  {'begin_rnn_3': [array([[ 1.06655514,  0.20870832],
       [-0.62020373, -0.92057413]], dtype=float32),
                   array([[-0.87163007,  0.17475815],
       [ 0.16320671,  0.86440241]], dtype=float32),
                   array([-0.09910914,  0.12928225], dtype=float32),
                   array([[ 0.88622832,  0.0257951 ],
       [-0.1858207 , -0.63723952]], dtype=float32),
                   array([ 0.29415649,  1.09380114], dtype=float32)]}],
 [{'end_input_11': []},
  {'end_rnn_3': [array([[ 1.06612265,  0.20919719],
       [-0.62004465, -0.9200052 ]], dtype=float32),
                 array([[-0.87126297,  0.17480175],
       [ 0.16317376,  0.8641423 ]], dtype=float32),
                 array([-0.09923106,  0.12970468], dtype=float32),
                 array([[ 0.88611281,  0.02598154],
       [-0.18512374, -0.63658291]], dtype=float32),
                 array([ 0.29480287,  1.09283495], dtype=float32)]}],
 [{'begin_input_11': []},
  {'begin_rnn_3'

## RNN Wrapper

### fullrnn

In [9]:
from keras.backend.tensorflow_backend import *


def fullrnn(step_function, inputs, initial_states,
        go_backwards=False, mask=None, constants=None,
        unroll=False, input_length=None):
    """Iterates over the time dimension of a tensor.

    # Arguments
        step_function: RNN step function.
            Parameters:
                inputs: tensor with shape `(samples, ...)` (no time dimension),
                    representing input for the batch of samples at a certain
                    time step.
                states: list of tensors.
            Returns:
                outputs: tensor with shape `(samples, output_dim)`
                    (no time dimension).
                new_states: list of tensors, same length and shapes
                    as 'states'. The first state in the list must be the
                    output tensor at the previous timestep.
        inputs: tensor of temporal data of shape `(samples, time, ...)`
            (at least 3D).
        initial_states: tensor with shape (samples, output_dim)
            (no time dimension),
            containing the initial values for the states used in
            the step function.
        go_backwards: boolean. If True, do the iteration over the time
            dimension in reverse order and return the reversed sequence.
        mask: binary tensor with shape `(samples, time, 1)`,
            with a zero for every element that is masked.
        constants: a list of constant values passed at each step.
        unroll: whether to unroll the RNN or to use a symbolic loop (`while_loop` or `scan` depending on backend).
        input_length: not relevant in the TensorFlow implementation.
            Must be specified if using unrolling with Theano.

    # Returns
        A tuple, `(last_output, outputs, new_states)`.

            last_output: the latest output of the rnn, of shape `(samples, ...)`
            outputs: tensor with shape `(samples, time, ...)` where each
                entry `outputs[s, t]` is the output of the step function
                at time `t` for sample `s`.
            new_states: list of tensors, latest states returned by
                the step function, of shape `(samples, ...)`.

    # Raises
        ValueError: if input dimension is less than 3.
        ValueError: if `unroll` is `True` but input timestep is not a fixed number.
        ValueError: if `mask` is provided (not `None`) but states is not provided
            (`len(states)` == 0).
    """
    ndim = len(inputs.get_shape())
    if ndim < 3:
        raise ValueError('Input should be at least 3D.')

    # Transpose to time-major, i.e.
    # from (batch, time, ...) to (time, batch, ...)
    axes = [1, 0] + list(range(2, ndim))
    inputs = tf.transpose(inputs, (axes))

    if mask is not None:
        if mask.dtype != tf.bool:
            mask = tf.cast(mask, tf.bool)
        if len(mask.get_shape()) == ndim - 1:
            mask = expand_dims(mask)
        mask = tf.transpose(mask, axes)

    if constants is None:
        constants = []

    global uses_learning_phase
    uses_learning_phase = False

    if unroll:
        if not inputs.get_shape()[0]:
            raise ValueError('Unrolling requires a '
                             'fixed number of timesteps.')
        states = initial_states
        successive_states = []
        successive_outputs = []

        input_list = tf.unstack(inputs)
        if go_backwards:
            input_list.reverse()

        if mask is not None:
            mask_list = tf.unstack(mask)
            if go_backwards:
                mask_list.reverse()

            for inp, mask_t in zip(input_list, mask_list):
                output, new_states = step_function(inp, states + constants)
                if getattr(output, '_uses_learning_phase', False):
                    uses_learning_phase = True

                # tf.where needs its condition tensor
                # to be the same shape as its two
                # result tensors, but in our case
                # the condition (mask) tensor is
                # (nsamples, 1), and A and B are (nsamples, ndimensions).
                # So we need to
                # broadcast the mask to match the shape of A and B.
                # That's what the tile call does,
                # it just repeats the mask along its second dimension
                # n times.
                tiled_mask_t = tf.tile(mask_t,
                                       tf.stack([1, tf.shape(output)[1]]))

                if not successive_outputs:
                    prev_output = zeros_like(output)
                else:
                    prev_output = successive_outputs[-1]

                output = tf.where(tiled_mask_t, output, prev_output)

                return_states = []
                for state, new_state in zip(states, new_states):
                    # (see earlier comment for tile explanation)
                    tiled_mask_t = tf.tile(mask_t,
                                           tf.stack([1, tf.shape(new_state)[1]]))
                    return_states.append(tf.where(tiled_mask_t,
                                                  new_state,
                                                  state))
                states = return_states
                successive_outputs.append(output)
                successive_states.append(states)
            last_output = successive_outputs[-1]
            new_states = successive_states[-1]
            outputs = tf.stack(successive_outputs)
        else:
            for inp in input_list:
                output, states = step_function(inp, states + constants)
                if getattr(output, '_uses_learning_phase', False):
                    uses_learning_phase = True
                successive_outputs.append(output)
                successive_states.append(states)
            last_output = successive_outputs[-1]
            new_states = successive_states[-1]
            outputs = tf.stack(successive_outputs)

    else:
        if go_backwards:
            inputs = reverse(inputs, 0)

        states = tuple(initial_states)

        time_steps = tf.shape(inputs)[0]
        outputs, _ = step_function(inputs[0], initial_states + constants)
        output_ta = tensor_array_ops.TensorArray(
            dtype=outputs.dtype,
            size=time_steps,
            tensor_array_name='output_ta')
        input_ta = tensor_array_ops.TensorArray(
            dtype=inputs.dtype,
            size=time_steps,
            tensor_array_name='input_ta')
        input_ta = input_ta.unstack(inputs)
        time = tf.constant(0, dtype='int32', name='time')

        if mask is not None:
            if not states:
                raise ValueError('No initial states provided! '
                                 'When using masking in an RNN, you should '
                                 'provide initial states '
                                 '(and your step function should return '
                                 'as its first state at time `t` '
                                 'the output at time `t-1`).')
            if go_backwards:
                mask = reverse(mask, 0)

            mask_ta = tensor_array_ops.TensorArray(
                dtype=tf.bool,
                size=time_steps,
                tensor_array_name='mask_ta')
            mask_ta = mask_ta.unstack(mask)

            def _step(time, output_ta_t, *states):
                """RNN step function.

                # Arguments
                    time: Current timestep value.
                    output_ta_t: TensorArray.
                    *states: List of states.

                # Returns
                    Tuple: `(time + 1,output_ta_t) + tuple(new_states)`
                """
                current_input = input_ta.read(time)

                # Edited Inputs
                full_inputs = input_ta.stack()  # [:time]


                mask_t = mask_ta.read(time)
                output, new_states = step_function(current_input,
                                                   tuple(states) +
                                                   tuple(constants))
                if getattr(output, '_uses_learning_phase', False):
                    global uses_learning_phase
                    uses_learning_phase = True
                for state, new_state in zip(states, new_states):
                    new_state.set_shape(state.get_shape())
                tiled_mask_t = tf.tile(mask_t,
                                       tf.stack([1, tf.shape(output)[1]]))
                output = tf.where(tiled_mask_t, output, states[0])
                new_states = [tf.where(tiled_mask_t, new_states[i], states[i]) for i in range(len(states))]
                output_ta_t = output_ta_t.write(time, output)
                return (time + 1, output_ta_t) + tuple(new_states)
        else:
            def _step(time, output_ta_t, *states):
                """RNN step function.

                # Arguments
                    time: Current timestep value.
                    output_ta_t: TensorArray.
                    *states: List of states.

                # Returns
                    Tuple: `(time + 1,output_ta_t) + tuple(new_states)`
                """
                current_input = input_ta.read(time)

                # Edited Inputs
                full_inputs = input_ta.stack()  # [:time]
                #states = states_ta_t.stack()[:time]


                output, new_states = step_function(current_input,
                                                   tuple(states) +
                                                   tuple(constants))
                if getattr(output, '_uses_learning_phase', False):
                    global uses_learning_phase
                    uses_learning_phase = True
                for state, new_state in zip(states, new_states):
                    new_state.set_shape(state.get_shape())
                output_ta_t = output_ta_t.write(time, output)
                return (time + 1, output_ta_t) + tuple(new_states)

        final_outputs = control_flow_ops.while_loop(
            cond=lambda time, *_: time < time_steps,
            body=_step,
            loop_vars=(time, output_ta) + states,
            parallel_iterations=32,
            swap_memory=True)
        last_time = final_outputs[0]
        output_ta = final_outputs[1]
        new_states = final_outputs[2:]

        outputs = output_ta.stack()
        last_output = output_ta.read(last_time - 1)

    axes = [1, 0] + list(range(2, len(outputs.get_shape())))
    outputs = tf.transpose(outputs, axes)
    last_output._uses_learning_phase = uses_learning_phase
    return last_output, outputs, new_states


### RNN Wrapper

In [43]:
class RNNWrapper(Layer):


    def __init__(self, cell,
                 return_sequences=False,
                 return_state=False,
                 go_backwards=False,
                 stateful=False,
                 unroll=False,
                 **kwargs):
        if isinstance(cell, (list, tuple)):
            cell = StackedRNNCells(cell)
        if not hasattr(cell, 'call'):
            raise ValueError('`cell` should have a `call` method. '
                             'The RNN was passed:', cell)
        if not hasattr(cell, 'state_size'):
            raise ValueError('The RNN cell should have '
                             'an attribute `state_size` '
                             '(tuple of integers, '
                             'one integer per RNN state).')
        super(RNNWrapper, self).__init__(**kwargs)
        self.cell = cell
        self.return_sequences = return_sequences
        self.return_state = return_state
        self.go_backwards = go_backwards
        self.stateful = stateful
        self.unroll = unroll

        self.supports_masking = True
        self.input_spec = [InputSpec(ndim=3)]
        self.state_spec = None
        self._states = None
        self.constants_spec = None
        self._num_constants = None

    @property
    def states(self):
        if self._states is None:
            if isinstance(self.cell.state_size, int):
                num_states = 1
            else:
                num_states = len(self.cell.state_size)
            return [None for _ in range(num_states)]
        return self._states

    @states.setter
    def states(self, states):
        self._states = states

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        if hasattr(self.cell.state_size, '__len__'):
            output_dim = self.cell.state_size[0]
        else:
            output_dim = self.cell.state_size

        if self.return_sequences:
            output_shape = (input_shape[0], input_shape[1], output_dim)
        else:
            output_shape = (input_shape[0], output_dim)

        if self.return_state:
            state_shape = [(input_shape[0], output_dim) for _ in self.states]
            return [output_shape] + state_shape
        else:
            return output_shape

    def compute_mask(self, inputs, mask):
        if isinstance(mask, list):
            mask = mask[0]
        output_mask = mask if self.return_sequences else None
        if self.return_state:
            state_mask = [None for _ in self.states]
            return [output_mask] + state_mask
        else:
            return output_mask

    def build(self, input_shape):
        # Note input_shape will be list of shapes of initial states and
        # constants if these are passed in __call__.
        if self._num_constants is not None:
            constants_shape = input_shape[-self._num_constants:]
        else:
            constants_shape = None

        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        batch_size = input_shape[0] if self.stateful else None
        input_dim = input_shape[-1]
        self.input_spec[0] = InputSpec(shape=(batch_size, None, input_dim))

        # allow cell (if layer) to build before we set or validate state_spec
        if isinstance(self.cell, Layer):
            step_input_shape = (input_shape[0],) + input_shape[2:]
            if constants_shape is not None:
                self.cell.build([step_input_shape] + constants_shape)
            else:
                self.cell.build(step_input_shape)

        # set or validate state_spec
        if hasattr(self.cell.state_size, '__len__'):
            state_size = list(self.cell.state_size)
        else:
            state_size = [self.cell.state_size]

        if self.state_spec is not None:
            # initial_state was passed in call, check compatibility
            if not [spec.shape[-1] for spec in self.state_spec] == state_size:
                raise ValueError(
                    'An initial_state was passed that is not compatible with '
                    '`cell.state_size`. Received `state_spec`={}; '
                    'However `cell.state_size` is '
                    '{}'.format(self.state_spec, self.cell.state_size))
        else:
            self.state_spec = [InputSpec(shape=(None, dim))
                               for dim in state_size]
        if self.stateful:
            self.reset_states()

    def get_initial_state(self, inputs):
        # build an all-zero tensor of shape (samples, output_dim)
        initial_state = K.zeros_like(inputs)  # (samples, timesteps, input_dim)
        initial_state = K.sum(initial_state, axis=(1, 2))  # (samples,)
        initial_state = K.expand_dims(initial_state)  # (samples, 1)
        if hasattr(self.cell.state_size, '__len__'):
            return [K.tile(initial_state, [1, dim])
                    for dim in self.cell.state_size]
        else:
            return [K.tile(initial_state, [1, self.cell.state_size])]

    def __call__(self, inputs, initial_state=None, constants=None, **kwargs):
        inputs, initial_state, constants = self._standardize_args(
            inputs, initial_state, constants)

        if initial_state is None and constants is None:
            return super(RNNWrapper, self).__call__(inputs, **kwargs)

        # If any of `initial_state` or `constants` are specified and are Keras
        # tensors, then add them to the inputs and temporarily modify the
        # input_spec to include them.

        additional_inputs = []
        additional_specs = []
        if initial_state is not None:
            kwargs['initial_state'] = initial_state
            additional_inputs += initial_state
            self.state_spec = [InputSpec(shape=K.int_shape(state))
                               for state in initial_state]
            additional_specs += self.state_spec
        if constants is not None:
            kwargs['constants'] = constants
            additional_inputs += constants
            self.constants_spec = [InputSpec(shape=K.int_shape(constant))
                                   for constant in constants]
            self._num_constants = len(constants)
            additional_specs += self.constants_spec
        # at this point additional_inputs cannot be empty
        is_keras_tensor = hasattr(additional_inputs[0], '_keras_history')
        for tensor in additional_inputs:
            if hasattr(tensor, '_keras_history') != is_keras_tensor:
                raise ValueError('The initial state or constants of an RNN'
                                 ' layer cannot be specified with a mix of'
                                 ' Keras tensors and non-Keras tensors')

        if is_keras_tensor:
            # Compute the full input spec, including state and constants
            full_input = [inputs] + additional_inputs
            full_input_spec = self.input_spec + additional_specs
            # Perform the call with temporarily replaced input_spec
            original_input_spec = self.input_spec
            self.input_spec = full_input_spec
            output = super(RNNWrapper, self).__call__(full_input, **kwargs)
            self.input_spec = original_input_spec
            return output
        else:
            return super(RNNWrapper, self).__call__(inputs, **kwargs)

    def get_attention(self, inputs):
        return inputs

    def call(self,
             inputs,
             mask=None,
             training=None,
             initial_state=None,
             constants=None):

        # input shape: `(samples, time (padded with zeros), input_dim)`
        # note that the .build() method of subclasses MUST define
        # self.input_spec and self.state_spec with complete input shapes.
        if isinstance(inputs, list):
            inputs = inputs[0]
        if initial_state is not None:
            pass
        elif self.stateful:
            initial_state = self.states
        else:
            initial_state = self.get_initial_state(inputs)

        if isinstance(mask, list):
            mask = mask[0]

        if len(initial_state) != len(self.states):
            raise ValueError('Layer has ' + str(len(self.states)) +
                             ' states but was passed ' +
                             str(len(initial_state)) +
                             ' initial states.')
        input_shape = K.int_shape(inputs)
        timesteps = input_shape[1]
        if self.unroll and timesteps in [None, 1]:
            raise ValueError('Cannot unroll a RNN if the '
                             'time dimension is undefined or equal to 1. \n'
                             '- If using a Sequential model, '
                             'specify the time dimension by passing '
                             'an `input_shape` or `batch_input_shape` '
                             'argument to your first layer. If your '
                             'first layer is an Embedding, you can '
                             'also use the `input_length` argument.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a `shape` '
                             'or `batch_shape` argument to your Input layer.')

        kwargs = {}
        if has_arg(self.cell.call, 'training'):
            kwargs['training'] = training

        if constants:
            if not has_arg(self.cell.call, 'constants'):
                raise ValueError('RNN cell does not support constants')

            def step(inputs, states):
                constants = states[-self._num_constants:]
                states = states[:-self._num_constants]
                return self.cell.call(inputs, states, constants=constants,
                                      **kwargs)
        else:
            def step(inputs, states):
                return self.cell.call(inputs, states, **kwargs)

        #constants = K.stack(inputs)
        constants = [inputs]
        last_output, outputs, states = K.rnn(step,
                                             inputs,
                                             initial_state,
                                             constants=constants,
                                             go_backwards=self.go_backwards,
                                             mask=mask,
                                             unroll=self.unroll,
                                             input_length=timesteps)
        if self.stateful:
            updates = []
            for i in range(len(states)):
                updates.append((self.states[i], states[i]))
            self.add_update(updates, inputs)

        if self.return_sequences:
            output = outputs
        else:
            output = last_output

        # Properly set learning phase
        if getattr(last_output, '_uses_learning_phase', False):
            output._uses_learning_phase = True

        if self.return_state:
            if not isinstance(states, (list, tuple)):
                states = [states]
            else:
                states = list(states)
            return [output] + states
        else:
            return output

    def _standardize_args(self, inputs, initial_state, constants):
        """Brings the arguments of `__call__` that can contain input tensors to
        standard format.

        When running a model loaded from file, the input tensors
        `initial_state` and `constants` can be passed to `RNN.__call__` as part
        of `inputs` instead of by the dedicated keyword arguments. This method
        makes sure the arguments are separated and that `initial_state` and
        `constants` are lists of tensors (or None).

        # Arguments
            inputs: tensor or list/tuple of tensors
            initial_state: tensor or list of tensors or None
            constants: tensor or list of tensors or None

        # Returns
            inputs: tensor
            initial_state: list of tensors or None
            constants: list of tensors or None
        """
        if isinstance(inputs, list):
            assert initial_state is None and constants is None
            if self._num_constants is not None:
                constants = inputs[-self._num_constants:]
                inputs = inputs[:-self._num_constants]
            if len(inputs) > 1:
                initial_state = inputs[1:]
            inputs = inputs[0]

        def to_list_or_none(x):
            if x is None or isinstance(x, list):
                return x
            if isinstance(x, tuple):
                return list(x)
            return [x]

        initial_state = to_list_or_none(initial_state)
        constants = to_list_or_none(constants)

        return inputs, initial_state, constants

    def reset_states(self, states=None):
        if not self.stateful:
            raise AttributeError('Layer must be stateful.')
        batch_size = self.input_spec[0].shape[0]
        if not batch_size:
            raise ValueError('If a RNN is stateful, it needs to know '
                             'its batch size. Specify the batch size '
                             'of your input tensors: \n'
                             '- If using a Sequential model, '
                             'specify the batch size by passing '
                             'a `batch_input_shape` '
                             'argument to your first layer.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a '
                             '`batch_shape` argument to your Input layer.')
        # initialize state if None
        if self.states[0] is None:
            if hasattr(self.cell.state_size, '__len__'):
                self.states = [K.zeros((batch_size, dim))
                               for dim in self.cell.state_size]
            else:
                self.states = [K.zeros((batch_size, self.cell.state_size))]
        elif states is None:
            if hasattr(self.cell.state_size, '__len__'):
                for state, dim in zip(self.states, self.cell.state_size):
                    K.set_value(state, np.zeros((batch_size, dim)))
            else:
                K.set_value(self.states[0],
                            np.zeros((batch_size, self.cell.state_size)))
        else:
            if not isinstance(states, (list, tuple)):
                states = [states]
            if len(states) != len(self.states):
                raise ValueError('Layer ' + self.name + ' expects ' +
                                 str(len(self.states)) + ' states, '
                                 'but it received ' + str(len(states)) +
                                 ' state values. Input received: ' +
                                 str(states))
            for index, (value, state) in enumerate(zip(states, self.states)):
                if hasattr(self.cell.state_size, '__len__'):
                    dim = self.cell.state_size[index]
                else:
                    dim = self.cell.state_size
                if value.shape != (batch_size, dim):
                    raise ValueError('State ' + str(index) +
                                     ' is incompatible with layer ' +
                                     self.name + ': expected shape=' +
                                     str((batch_size, dim)) +
                                     ', found shape=' + str(value.shape))
                # TODO: consider batch calls to `set_value`.
                K.set_value(state, value)

    def get_config(self):
        config = {'return_sequences': self.return_sequences,
                  'return_state': self.return_state,
                  'go_backwards': self.go_backwards,
                  'stateful': self.stateful,
                  'unroll': self.unroll}
        if self._num_constants is not None:
            config['num_constants'] = self._num_constants

        cell_config = self.cell.get_config()
        config['cell'] = {'class_name': self.cell.__class__.__name__,
                          'config': cell_config}
        base_config = super(RNNWrapper, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config, custom_objects=None):
        from . import deserialize as deserialize_layer
        cell = deserialize_layer(config.pop('cell'),
                                 custom_objects=custom_objects)
        num_constants = config.pop('num_constants', None)
        layer = cls(cell, **config)
        layer._num_constants = num_constants
        return layer

    @property
    def trainable_weights(self):
        if not self.trainable:
            return []
        if isinstance(self.cell, Layer):
            return self.cell.trainable_weights
        return []

    @property
    def non_trainable_weights(self):
        if isinstance(self.cell, Layer):
            if not self.trainable:
                return self.cell.weights
            return self.cell.non_trainable_weights
        return []

    @property
    def losses(self):
        if isinstance(self.cell, Layer):
            return self.cell.losses
        return []

    def get_losses_for(self, inputs=None):
        if isinstance(self.cell, Layer):
            cell_losses = self.cell.get_losses_for(inputs)
            return cell_losses + super(RNNWrapper, self).get_losses_for(inputs)
        return super(RNNWrapper, self).get_losses_for(inputs)


In [24]:
aa = Input(shape=(3, 2), dtype='float32')
bb = SimpleRNNCell(2)
cc = CellWrapper(bb)
dd = RNN(cc, return_sequences=True, return_state=False)(aa)
ee = Model(inputs=aa, outputs=dd)
ee.compile(optimizer='adam', loss='mae', metrics=['mae'])
ee.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3, 2)              0         
_________________________________________________________________
rnn_3 (RNN)                  (None, 3, 2)              16        
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


#### Training

In [41]:
print('EPOCH_NUM: %s, BATCH_SIZE %s' % (EPOCH_NUM, BATCH_SIZE))

model = ee
fitted = model.fit(train_X, train_Y,
                   epochs=10,     # How many times to run back_propagation
                   batch_size=2,  # How many data to deal with at one epoch
                   validation_split=0.2,
                   verbose=2,       # 1: progress bar, 2: one line per epoch
                   #validation_data=(testX, testY),  # Validation set
                   shuffle=True,
                   callbacks=[history],
                  )

# Save model
model.save('gru_attention_embedding_model.h5')

EPOCH_NUM: 100, BATCH_SIZE 256
Train on 40 samples, validate on 10 samples
Epoch 1/10
 - 0s - loss: 0.4883 - mean_absolute_error: 0.4883 - val_loss: 0.4634 - val_mean_absolute_error: 0.4634
Epoch 2/10
 - 0s - loss: 0.4962 - mean_absolute_error: 0.4962 - val_loss: 0.4515 - val_mean_absolute_error: 0.4515
Epoch 3/10
 - 0s - loss: 0.4707 - mean_absolute_error: 0.4707 - val_loss: 0.4395 - val_mean_absolute_error: 0.4395
Epoch 4/10
 - 0s - loss: 0.4717 - mean_absolute_error: 0.4717 - val_loss: 0.4276 - val_mean_absolute_error: 0.4276
Epoch 5/10
 - 0s - loss: 0.4313 - mean_absolute_error: 0.4313 - val_loss: 0.4182 - val_mean_absolute_error: 0.4182
Epoch 6/10
 - 0s - loss: 0.3984 - mean_absolute_error: 0.3984 - val_loss: 0.4073 - val_mean_absolute_error: 0.4073
Epoch 7/10
 - 0s - loss: 0.4118 - mean_absolute_error: 0.4118 - val_loss: 0.3973 - val_mean_absolute_error: 0.3973
Epoch 8/10
 - 0s - loss: 0.4204 - mean_absolute_error: 0.4204 - val_loss: 0.3904 - val_mean_absolute_error: 0.3904
Epoch

In [42]:
pprint(history.weights[:4])

[[{'begin_input_11': []},
  {'begin_rnn_3': [array([[ 1.06655514,  0.20870832],
       [-0.62020373, -0.92057413]], dtype=float32),
                   array([[-0.87163007,  0.17475815],
       [ 0.16320671,  0.86440241]], dtype=float32),
                   array([-0.09910914,  0.12928225], dtype=float32),
                   array([[ 0.88622832,  0.0257951 ],
       [-0.1858207 , -0.63723952]], dtype=float32),
                   array([ 0.29415649,  1.09380114], dtype=float32)]}],
 [{'end_input_11': []},
  {'end_rnn_3': [array([[ 1.06612265,  0.20919719],
       [-0.62004465, -0.9200052 ]], dtype=float32),
                 array([[-0.87126297,  0.17480175],
       [ 0.16317376,  0.8641423 ]], dtype=float32),
                 array([-0.09923106,  0.12970468], dtype=float32),
                 array([[ 0.88611281,  0.02598154],
       [-0.18512374, -0.63658291]], dtype=float32),
                 array([ 0.29480287,  1.09283495], dtype=float32)]}],
 [{'begin_input_11': []},
  {'begin_rnn_3'

### ATTRNN

In [43]:
class ATTRNN(Layer):


    def __init__(self, cell,
                 return_sequences=False,
                 return_state=False,
                 go_backwards=False,
                 stateful=False,
                 unroll=False,
                 **kwargs):
        if isinstance(cell, (list, tuple)):
            cell = StackedRNNCells(cell)
        if not hasattr(cell, 'call'):
            raise ValueError('`cell` should have a `call` method. '
                             'The RNN was passed:', cell)
        if not hasattr(cell, 'state_size'):
            raise ValueError('The RNN cell should have '
                             'an attribute `state_size` '
                             '(tuple of integers, '
                             'one integer per RNN state).')
        super(ATTRNN, self).__init__(**kwargs)
        self.cell = cell
        self.return_sequences = return_sequences
        self.return_state = return_state
        self.go_backwards = go_backwards
        self.stateful = stateful
        self.unroll = unroll

        self.supports_masking = True
        self.input_spec = [InputSpec(ndim=3)]
        self.state_spec = None
        self._states = None
        self.constants_spec = None
        self._num_constants = None

    @property
    def states(self):
        if self._states is None:
            if isinstance(self.cell.state_size, int):
                num_states = 1
            else:
                num_states = len(self.cell.state_size)
            return [None for _ in range(num_states)]
        return self._states

    @states.setter
    def states(self, states):
        self._states = states

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        if hasattr(self.cell.state_size, '__len__'):
            output_dim = self.cell.state_size[0]
        else:
            output_dim = self.cell.state_size

        if self.return_sequences:
            output_shape = (input_shape[0], input_shape[1], output_dim)
        else:
            output_shape = (input_shape[0], output_dim)

        if self.return_state:
            state_shape = [(input_shape[0], output_dim) for _ in self.states]
            return [output_shape] + state_shape
        else:
            return output_shape

    def compute_mask(self, inputs, mask):
        if isinstance(mask, list):
            mask = mask[0]
        output_mask = mask if self.return_sequences else None
        if self.return_state:
            state_mask = [None for _ in self.states]
            return [output_mask] + state_mask
        else:
            return output_mask

    def build(self, input_shape):
        # Note input_shape will be list of shapes of initial states and
        # constants if these are passed in __call__.
        if self._num_constants is not None:
            constants_shape = input_shape[-self._num_constants:]
        else:
            constants_shape = None

        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        batch_size = input_shape[0] if self.stateful else None
        input_dim = input_shape[-1]
        self.input_spec[0] = InputSpec(shape=(batch_size, None, input_dim))

        # allow cell (if layer) to build before we set or validate state_spec
        if isinstance(self.cell, Layer):
            step_input_shape = (input_shape[0],) + input_shape[2:]
            if constants_shape is not None:
                self.cell.build([step_input_shape] + constants_shape)
            else:
                self.cell.build(step_input_shape)

        # set or validate state_spec
        if hasattr(self.cell.state_size, '__len__'):
            state_size = list(self.cell.state_size)
        else:
            state_size = [self.cell.state_size]

        if self.state_spec is not None:
            # initial_state was passed in call, check compatibility
            if not [spec.shape[-1] for spec in self.state_spec] == state_size:
                raise ValueError(
                    'An initial_state was passed that is not compatible with '
                    '`cell.state_size`. Received `state_spec`={}; '
                    'However `cell.state_size` is '
                    '{}'.format(self.state_spec, self.cell.state_size))
        else:
            self.state_spec = [InputSpec(shape=(None, dim))
                               for dim in state_size]
        if self.stateful:
            self.reset_states()

    def get_initial_state(self, inputs):
        # build an all-zero tensor of shape (samples, output_dim)
        initial_state = K.zeros_like(inputs)  # (samples, timesteps, input_dim)
        initial_state = K.sum(initial_state, axis=(1, 2))  # (samples,)
        initial_state = K.expand_dims(initial_state)  # (samples, 1)
        if hasattr(self.cell.state_size, '__len__'):
            return [K.tile(initial_state, [1, dim])
                    for dim in self.cell.state_size]
        else:
            return [K.tile(initial_state, [1, self.cell.state_size])]

    def __call__(self, inputs, initial_state=None, constants=None, **kwargs):
        inputs, initial_state, constants = self._standardize_args(
            inputs, initial_state, constants)

        if initial_state is None and constants is None:
            return super(ATTRNN, self).__call__(inputs, **kwargs)

        # If any of `initial_state` or `constants` are specified and are Keras
        # tensors, then add them to the inputs and temporarily modify the
        # input_spec to include them.

        additional_inputs = []
        additional_specs = []
        if initial_state is not None:
            kwargs['initial_state'] = initial_state
            additional_inputs += initial_state
            self.state_spec = [InputSpec(shape=K.int_shape(state))
                               for state in initial_state]
            additional_specs += self.state_spec
        if constants is not None:
            kwargs['constants'] = constants
            additional_inputs += constants
            self.constants_spec = [InputSpec(shape=K.int_shape(constant))
                                   for constant in constants]
            self._num_constants = len(constants)
            additional_specs += self.constants_spec
        # at this point additional_inputs cannot be empty
        is_keras_tensor = hasattr(additional_inputs[0], '_keras_history')
        for tensor in additional_inputs:
            if hasattr(tensor, '_keras_history') != is_keras_tensor:
                raise ValueError('The initial state or constants of an RNN'
                                 ' layer cannot be specified with a mix of'
                                 ' Keras tensors and non-Keras tensors')

        if is_keras_tensor:
            # Compute the full input spec, including state and constants
            full_input = [inputs] + additional_inputs
            full_input_spec = self.input_spec + additional_specs
            # Perform the call with temporarily replaced input_spec
            original_input_spec = self.input_spec
            self.input_spec = full_input_spec
            output = super(ATTRNN, self).__call__(full_input, **kwargs)
            self.input_spec = original_input_spec
            return output
        else:
            return super(ATTRNN, self).__call__(inputs, **kwargs)

    def get_attention(self, inputs):
        return inputs

    def call(self,
             inputs,
             mask=None,
             training=None,
             initial_state=None,
             constants=None):

        # input shape: `(samples, time (padded with zeros), input_dim)`
        # note that the .build() method of subclasses MUST define
        # self.input_spec and self.state_spec with complete input shapes.
        if isinstance(inputs, list):
            inputs = inputs[0]
        if initial_state is not None:
            pass
        elif self.stateful:
            initial_state = self.states
        else:
            initial_state = self.get_initial_state(inputs)

        if isinstance(mask, list):
            mask = mask[0]

        if len(initial_state) != len(self.states):
            raise ValueError('Layer has ' + str(len(self.states)) +
                             ' states but was passed ' +
                             str(len(initial_state)) +
                             ' initial states.')
        input_shape = K.int_shape(inputs)
        timesteps = input_shape[1]
        if self.unroll and timesteps in [None, 1]:
            raise ValueError('Cannot unroll a RNN if the '
                             'time dimension is undefined or equal to 1. \n'
                             '- If using a Sequential model, '
                             'specify the time dimension by passing '
                             'an `input_shape` or `batch_input_shape` '
                             'argument to your first layer. If your '
                             'first layer is an Embedding, you can '
                             'also use the `input_length` argument.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a `shape` '
                             'or `batch_shape` argument to your Input layer.')

        kwargs = {}
        if has_arg(self.cell.call, 'training'):
            kwargs['training'] = training

        if constants:
            if not has_arg(self.cell.call, 'constants'):
                raise ValueError('RNN cell does not support constants')

            def step(inputs, states):
                constants = states[-self._num_constants:]
                states = states[:-self._num_constants]
                return self.cell.call(inputs, states, constants=constants,
                                      **kwargs)
        else:
            def step(inputs, states):
                return self.cell.call(inputs, states, **kwargs)

        #constants = K.stack(inputs)
        constants = [inputs]
        last_output, outputs, states = K.rnn(step,
                                             inputs,
                                             initial_state,
                                             constants=constants,
                                             go_backwards=self.go_backwards,
                                             mask=mask,
                                             unroll=self.unroll,
                                             input_length=timesteps)
        if self.stateful:
            updates = []
            for i in range(len(states)):
                updates.append((self.states[i], states[i]))
            self.add_update(updates, inputs)

        if self.return_sequences:
            output = outputs
        else:
            output = last_output

        # Properly set learning phase
        if getattr(last_output, '_uses_learning_phase', False):
            output._uses_learning_phase = True

        if self.return_state:
            if not isinstance(states, (list, tuple)):
                states = [states]
            else:
                states = list(states)
            return [output] + states
        else:
            return output

    def _standardize_args(self, inputs, initial_state, constants):
        """Brings the arguments of `__call__` that can contain input tensors to
        standard format.

        When running a model loaded from file, the input tensors
        `initial_state` and `constants` can be passed to `RNN.__call__` as part
        of `inputs` instead of by the dedicated keyword arguments. This method
        makes sure the arguments are separated and that `initial_state` and
        `constants` are lists of tensors (or None).

        # Arguments
            inputs: tensor or list/tuple of tensors
            initial_state: tensor or list of tensors or None
            constants: tensor or list of tensors or None

        # Returns
            inputs: tensor
            initial_state: list of tensors or None
            constants: list of tensors or None
        """
        if isinstance(inputs, list):
            assert initial_state is None and constants is None
            if self._num_constants is not None:
                constants = inputs[-self._num_constants:]
                inputs = inputs[:-self._num_constants]
            if len(inputs) > 1:
                initial_state = inputs[1:]
            inputs = inputs[0]

        def to_list_or_none(x):
            if x is None or isinstance(x, list):
                return x
            if isinstance(x, tuple):
                return list(x)
            return [x]

        initial_state = to_list_or_none(initial_state)
        constants = to_list_or_none(constants)

        return inputs, initial_state, constants

    def reset_states(self, states=None):
        if not self.stateful:
            raise AttributeError('Layer must be stateful.')
        batch_size = self.input_spec[0].shape[0]
        if not batch_size:
            raise ValueError('If a RNN is stateful, it needs to know '
                             'its batch size. Specify the batch size '
                             'of your input tensors: \n'
                             '- If using a Sequential model, '
                             'specify the batch size by passing '
                             'a `batch_input_shape` '
                             'argument to your first layer.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a '
                             '`batch_shape` argument to your Input layer.')
        # initialize state if None
        if self.states[0] is None:
            if hasattr(self.cell.state_size, '__len__'):
                self.states = [K.zeros((batch_size, dim))
                               for dim in self.cell.state_size]
            else:
                self.states = [K.zeros((batch_size, self.cell.state_size))]
        elif states is None:
            if hasattr(self.cell.state_size, '__len__'):
                for state, dim in zip(self.states, self.cell.state_size):
                    K.set_value(state, np.zeros((batch_size, dim)))
            else:
                K.set_value(self.states[0],
                            np.zeros((batch_size, self.cell.state_size)))
        else:
            if not isinstance(states, (list, tuple)):
                states = [states]
            if len(states) != len(self.states):
                raise ValueError('Layer ' + self.name + ' expects ' +
                                 str(len(self.states)) + ' states, '
                                 'but it received ' + str(len(states)) +
                                 ' state values. Input received: ' +
                                 str(states))
            for index, (value, state) in enumerate(zip(states, self.states)):
                if hasattr(self.cell.state_size, '__len__'):
                    dim = self.cell.state_size[index]
                else:
                    dim = self.cell.state_size
                if value.shape != (batch_size, dim):
                    raise ValueError('State ' + str(index) +
                                     ' is incompatible with layer ' +
                                     self.name + ': expected shape=' +
                                     str((batch_size, dim)) +
                                     ', found shape=' + str(value.shape))
                # TODO: consider batch calls to `set_value`.
                K.set_value(state, value)

    def get_config(self):
        config = {'return_sequences': self.return_sequences,
                  'return_state': self.return_state,
                  'go_backwards': self.go_backwards,
                  'stateful': self.stateful,
                  'unroll': self.unroll}
        if self._num_constants is not None:
            config['num_constants'] = self._num_constants

        cell_config = self.cell.get_config()
        config['cell'] = {'class_name': self.cell.__class__.__name__,
                          'config': cell_config}
        base_config = super(ATTRNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config, custom_objects=None):
        from . import deserialize as deserialize_layer
        cell = deserialize_layer(config.pop('cell'),
                                 custom_objects=custom_objects)
        num_constants = config.pop('num_constants', None)
        layer = cls(cell, **config)
        layer._num_constants = num_constants
        return layer

    @property
    def trainable_weights(self):
        if not self.trainable:
            return []
        if isinstance(self.cell, Layer):
            return self.cell.trainable_weights
        return []

    @property
    def non_trainable_weights(self):
        if isinstance(self.cell, Layer):
            if not self.trainable:
                return self.cell.weights
            return self.cell.non_trainable_weights
        return []

    @property
    def losses(self):
        if isinstance(self.cell, Layer):
            return self.cell.losses
        return []

    def get_losses_for(self, inputs=None):
        if isinstance(self.cell, Layer):
            cell_losses = self.cell.get_losses_for(inputs)
            return cell_losses + super(ATTRNN, self).get_losses_for(inputs)
        return super(ATTRNN, self).get_losses_for(inputs)


In [24]:
aa = Input(shape=(3, 2), dtype='float32')
bb = SimpleRNNCell(2)
cc = CellWrapper(bb)
dd = ATTRNNWrapper(cc, return_sequences=True, return_state=False)(aa)
ee = Model(inputs=aa, outputs=dd)
ee.compile(optimizer='adam', loss='mae', metrics=['mae'])
ee.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3, 2)              0         
_________________________________________________________________
rnn_3 (RNN)                  (None, 3, 2)              16        
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


#### Training

In [41]:
print('EPOCH_NUM: %s, BATCH_SIZE %s' % (EPOCH_NUM, BATCH_SIZE))

model = ee
fitted = model.fit(train_X, train_Y,
                   epochs=10,     # How many times to run back_propagation
                   batch_size=2,  # How many data to deal with at one epoch
                   validation_split=0.2,
                   verbose=2,       # 1: progress bar, 2: one line per epoch
                   #validation_data=(testX, testY),  # Validation set
                   shuffle=True,
                   callbacks=[history],
                  )

# Save model
model.save('gru_attention_embedding_model.h5')

EPOCH_NUM: 100, BATCH_SIZE 256
Train on 40 samples, validate on 10 samples
Epoch 1/10
 - 0s - loss: 0.4883 - mean_absolute_error: 0.4883 - val_loss: 0.4634 - val_mean_absolute_error: 0.4634
Epoch 2/10
 - 0s - loss: 0.4962 - mean_absolute_error: 0.4962 - val_loss: 0.4515 - val_mean_absolute_error: 0.4515
Epoch 3/10
 - 0s - loss: 0.4707 - mean_absolute_error: 0.4707 - val_loss: 0.4395 - val_mean_absolute_error: 0.4395
Epoch 4/10
 - 0s - loss: 0.4717 - mean_absolute_error: 0.4717 - val_loss: 0.4276 - val_mean_absolute_error: 0.4276
Epoch 5/10
 - 0s - loss: 0.4313 - mean_absolute_error: 0.4313 - val_loss: 0.4182 - val_mean_absolute_error: 0.4182
Epoch 6/10
 - 0s - loss: 0.3984 - mean_absolute_error: 0.3984 - val_loss: 0.4073 - val_mean_absolute_error: 0.4073
Epoch 7/10
 - 0s - loss: 0.4118 - mean_absolute_error: 0.4118 - val_loss: 0.3973 - val_mean_absolute_error: 0.3973
Epoch 8/10
 - 0s - loss: 0.4204 - mean_absolute_error: 0.4204 - val_loss: 0.3904 - val_mean_absolute_error: 0.3904
Epoch

In [42]:
pprint(history.weights[:4])

[[{'begin_input_11': []},
  {'begin_rnn_3': [array([[ 1.06655514,  0.20870832],
       [-0.62020373, -0.92057413]], dtype=float32),
                   array([[-0.87163007,  0.17475815],
       [ 0.16320671,  0.86440241]], dtype=float32),
                   array([-0.09910914,  0.12928225], dtype=float32),
                   array([[ 0.88622832,  0.0257951 ],
       [-0.1858207 , -0.63723952]], dtype=float32),
                   array([ 0.29415649,  1.09380114], dtype=float32)]}],
 [{'end_input_11': []},
  {'end_rnn_3': [array([[ 1.06612265,  0.20919719],
       [-0.62004465, -0.9200052 ]], dtype=float32),
                 array([[-0.87126297,  0.17480175],
       [ 0.16317376,  0.8641423 ]], dtype=float32),
                 array([-0.09923106,  0.12970468], dtype=float32),
                 array([[ 0.88611281,  0.02598154],
       [-0.18512374, -0.63658291]], dtype=float32),
                 array([ 0.29480287,  1.09283495], dtype=float32)]}],
 [{'begin_input_11': []},
  {'begin_rnn_3'

### ATTRNN Wrapper

In [43]:
class ATTRNNWrapper(Layer):


    def __init__(self, cell,
                 return_sequences=False,
                 return_state=False,
                 go_backwards=False,
                 stateful=False,
                 unroll=False,
                 **kwargs):
        if isinstance(cell, (list, tuple)):
            cell = StackedRNNCells(cell)
        if not hasattr(cell, 'call'):
            raise ValueError('`cell` should have a `call` method. '
                             'The RNN was passed:', cell)
        if not hasattr(cell, 'state_size'):
            raise ValueError('The RNN cell should have '
                             'an attribute `state_size` '
                             '(tuple of integers, '
                             'one integer per RNN state).')
        super(ATTRNNWrapper, self).__init__(**kwargs)
        self.cell = cell
        self.return_sequences = return_sequences
        self.return_state = return_state
        self.go_backwards = go_backwards
        self.stateful = stateful
        self.unroll = unroll

        self.supports_masking = True
        self.input_spec = [InputSpec(ndim=3)]
        self.state_spec = None
        self._states = None
        self.constants_spec = None
        self._num_constants = None

    @property
    def states(self):
        if self._states is None:
            if isinstance(self.cell.state_size, int):
                num_states = 1
            else:
                num_states = len(self.cell.state_size)
            return [None for _ in range(num_states)]
        return self._states

    @states.setter
    def states(self, states):
        self._states = states

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        if hasattr(self.cell.state_size, '__len__'):
            output_dim = self.cell.state_size[0]
        else:
            output_dim = self.cell.state_size

        if self.return_sequences:
            output_shape = (input_shape[0], input_shape[1], output_dim)
        else:
            output_shape = (input_shape[0], output_dim)

        if self.return_state:
            state_shape = [(input_shape[0], output_dim) for _ in self.states]
            return [output_shape] + state_shape
        else:
            return output_shape

    def compute_mask(self, inputs, mask):
        if isinstance(mask, list):
            mask = mask[0]
        output_mask = mask if self.return_sequences else None
        if self.return_state:
            state_mask = [None for _ in self.states]
            return [output_mask] + state_mask
        else:
            return output_mask

    def build(self, input_shape):
        # Note input_shape will be list of shapes of initial states and
        # constants if these are passed in __call__.
        if self._num_constants is not None:
            constants_shape = input_shape[-self._num_constants:]
        else:
            constants_shape = None

        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        batch_size = input_shape[0] if self.stateful else None
        input_dim = input_shape[-1]
        self.input_spec[0] = InputSpec(shape=(batch_size, None, input_dim))

        # allow cell (if layer) to build before we set or validate state_spec
        if isinstance(self.cell, Layer):
            step_input_shape = (input_shape[0],) + input_shape[2:]
            if constants_shape is not None:
                self.cell.build([step_input_shape] + constants_shape)
            else:
                self.cell.build(step_input_shape)

        # set or validate state_spec
        if hasattr(self.cell.state_size, '__len__'):
            state_size = list(self.cell.state_size)
        else:
            state_size = [self.cell.state_size]

        if self.state_spec is not None:
            # initial_state was passed in call, check compatibility
            if not [spec.shape[-1] for spec in self.state_spec] == state_size:
                raise ValueError(
                    'An initial_state was passed that is not compatible with '
                    '`cell.state_size`. Received `state_spec`={}; '
                    'However `cell.state_size` is '
                    '{}'.format(self.state_spec, self.cell.state_size))
        else:
            self.state_spec = [InputSpec(shape=(None, dim))
                               for dim in state_size]
        if self.stateful:
            self.reset_states()

    def get_initial_state(self, inputs):
        # build an all-zero tensor of shape (samples, output_dim)
        initial_state = K.zeros_like(inputs)  # (samples, timesteps, input_dim)
        initial_state = K.sum(initial_state, axis=(1, 2))  # (samples,)
        initial_state = K.expand_dims(initial_state)  # (samples, 1)
        if hasattr(self.cell.state_size, '__len__'):
            return [K.tile(initial_state, [1, dim])
                    for dim in self.cell.state_size]
        else:
            return [K.tile(initial_state, [1, self.cell.state_size])]

    def __call__(self, inputs, initial_state=None, constants=None, **kwargs):
        inputs, initial_state, constants = self._standardize_args(
            inputs, initial_state, constants)

        if initial_state is None and constants is None:
            return super(ATTRNNWrapper, self).__call__(inputs, **kwargs)

        # If any of `initial_state` or `constants` are specified and are Keras
        # tensors, then add them to the inputs and temporarily modify the
        # input_spec to include them.

        additional_inputs = []
        additional_specs = []
        if initial_state is not None:
            kwargs['initial_state'] = initial_state
            additional_inputs += initial_state
            self.state_spec = [InputSpec(shape=K.int_shape(state))
                               for state in initial_state]
            additional_specs += self.state_spec
        if constants is not None:
            kwargs['constants'] = constants
            additional_inputs += constants
            self.constants_spec = [InputSpec(shape=K.int_shape(constant))
                                   for constant in constants]
            self._num_constants = len(constants)
            additional_specs += self.constants_spec
        # at this point additional_inputs cannot be empty
        is_keras_tensor = hasattr(additional_inputs[0], '_keras_history')
        for tensor in additional_inputs:
            if hasattr(tensor, '_keras_history') != is_keras_tensor:
                raise ValueError('The initial state or constants of an RNN'
                                 ' layer cannot be specified with a mix of'
                                 ' Keras tensors and non-Keras tensors')

        if is_keras_tensor:
            # Compute the full input spec, including state and constants
            full_input = [inputs] + additional_inputs
            full_input_spec = self.input_spec + additional_specs
            # Perform the call with temporarily replaced input_spec
            original_input_spec = self.input_spec
            self.input_spec = full_input_spec
            output = super(ATTRNNWrapper, self).__call__(full_input, **kwargs)
            self.input_spec = original_input_spec
            return output
        else:
            return super(ATTRNNWrapper, self).__call__(inputs, **kwargs)

    def get_attention(self, inputs):
        return inputs

    def call(self,
             inputs,
             mask=None,
             training=None,
             initial_state=None,
             constants=None):

        # input shape: `(samples, time (padded with zeros), input_dim)`
        # note that the .build() method of subclasses MUST define
        # self.input_spec and self.state_spec with complete input shapes.
        if isinstance(inputs, list):
            inputs = inputs[0]
        if initial_state is not None:
            pass
        elif self.stateful:
            initial_state = self.states
        else:
            initial_state = self.get_initial_state(inputs)

        if isinstance(mask, list):
            mask = mask[0]

        if len(initial_state) != len(self.states):
            raise ValueError('Layer has ' + str(len(self.states)) +
                             ' states but was passed ' +
                             str(len(initial_state)) +
                             ' initial states.')
        input_shape = K.int_shape(inputs)
        timesteps = input_shape[1]
        if self.unroll and timesteps in [None, 1]:
            raise ValueError('Cannot unroll a RNN if the '
                             'time dimension is undefined or equal to 1. \n'
                             '- If using a Sequential model, '
                             'specify the time dimension by passing '
                             'an `input_shape` or `batch_input_shape` '
                             'argument to your first layer. If your '
                             'first layer is an Embedding, you can '
                             'also use the `input_length` argument.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a `shape` '
                             'or `batch_shape` argument to your Input layer.')

        kwargs = {}
        if has_arg(self.cell.call, 'training'):
            kwargs['training'] = training

        if constants:
            if not has_arg(self.cell.call, 'constants'):
                raise ValueError('RNN cell does not support constants')

            def step(inputs, states):
                constants = states[-self._num_constants:]
                states = states[:-self._num_constants]
                return self.cell.call(inputs, states, constants=constants,
                                      **kwargs)
        else:
            def step(inputs, states):
                return self.cell.call(inputs, states, **kwargs)

        #constants = K.stack(inputs)
        constants = [inputs]
        last_output, outputs, states = K.rnn(step,
                                             inputs,
                                             initial_state,
                                             constants=constants,
                                             go_backwards=self.go_backwards,
                                             mask=mask,
                                             unroll=self.unroll,
                                             input_length=timesteps)
        if self.stateful:
            updates = []
            for i in range(len(states)):
                updates.append((self.states[i], states[i]))
            self.add_update(updates, inputs)

        if self.return_sequences:
            output = outputs
        else:
            output = last_output

        # Properly set learning phase
        if getattr(last_output, '_uses_learning_phase', False):
            output._uses_learning_phase = True

        if self.return_state:
            if not isinstance(states, (list, tuple)):
                states = [states]
            else:
                states = list(states)
            return [output] + states
        else:
            return output

    def _standardize_args(self, inputs, initial_state, constants):
        """Brings the arguments of `__call__` that can contain input tensors to
        standard format.

        When running a model loaded from file, the input tensors
        `initial_state` and `constants` can be passed to `RNN.__call__` as part
        of `inputs` instead of by the dedicated keyword arguments. This method
        makes sure the arguments are separated and that `initial_state` and
        `constants` are lists of tensors (or None).

        # Arguments
            inputs: tensor or list/tuple of tensors
            initial_state: tensor or list of tensors or None
            constants: tensor or list of tensors or None

        # Returns
            inputs: tensor
            initial_state: list of tensors or None
            constants: list of tensors or None
        """
        if isinstance(inputs, list):
            assert initial_state is None and constants is None
            if self._num_constants is not None:
                constants = inputs[-self._num_constants:]
                inputs = inputs[:-self._num_constants]
            if len(inputs) > 1:
                initial_state = inputs[1:]
            inputs = inputs[0]

        def to_list_or_none(x):
            if x is None or isinstance(x, list):
                return x
            if isinstance(x, tuple):
                return list(x)
            return [x]

        initial_state = to_list_or_none(initial_state)
        constants = to_list_or_none(constants)

        return inputs, initial_state, constants

    def reset_states(self, states=None):
        if not self.stateful:
            raise AttributeError('Layer must be stateful.')
        batch_size = self.input_spec[0].shape[0]
        if not batch_size:
            raise ValueError('If a RNN is stateful, it needs to know '
                             'its batch size. Specify the batch size '
                             'of your input tensors: \n'
                             '- If using a Sequential model, '
                             'specify the batch size by passing '
                             'a `batch_input_shape` '
                             'argument to your first layer.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a '
                             '`batch_shape` argument to your Input layer.')
        # initialize state if None
        if self.states[0] is None:
            if hasattr(self.cell.state_size, '__len__'):
                self.states = [K.zeros((batch_size, dim))
                               for dim in self.cell.state_size]
            else:
                self.states = [K.zeros((batch_size, self.cell.state_size))]
        elif states is None:
            if hasattr(self.cell.state_size, '__len__'):
                for state, dim in zip(self.states, self.cell.state_size):
                    K.set_value(state, np.zeros((batch_size, dim)))
            else:
                K.set_value(self.states[0],
                            np.zeros((batch_size, self.cell.state_size)))
        else:
            if not isinstance(states, (list, tuple)):
                states = [states]
            if len(states) != len(self.states):
                raise ValueError('Layer ' + self.name + ' expects ' +
                                 str(len(self.states)) + ' states, '
                                 'but it received ' + str(len(states)) +
                                 ' state values. Input received: ' +
                                 str(states))
            for index, (value, state) in enumerate(zip(states, self.states)):
                if hasattr(self.cell.state_size, '__len__'):
                    dim = self.cell.state_size[index]
                else:
                    dim = self.cell.state_size
                if value.shape != (batch_size, dim):
                    raise ValueError('State ' + str(index) +
                                     ' is incompatible with layer ' +
                                     self.name + ': expected shape=' +
                                     str((batch_size, dim)) +
                                     ', found shape=' + str(value.shape))
                # TODO: consider batch calls to `set_value`.
                K.set_value(state, value)

    def get_config(self):
        config = {'return_sequences': self.return_sequences,
                  'return_state': self.return_state,
                  'go_backwards': self.go_backwards,
                  'stateful': self.stateful,
                  'unroll': self.unroll}
        if self._num_constants is not None:
            config['num_constants'] = self._num_constants

        cell_config = self.cell.get_config()
        config['cell'] = {'class_name': self.cell.__class__.__name__,
                          'config': cell_config}
        base_config = super(ATTRNNWrapper, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config, custom_objects=None):
        from . import deserialize as deserialize_layer
        cell = deserialize_layer(config.pop('cell'),
                                 custom_objects=custom_objects)
        num_constants = config.pop('num_constants', None)
        layer = cls(cell, **config)
        layer._num_constants = num_constants
        return layer

    @property
    def trainable_weights(self):
        if not self.trainable:
            return []
        if isinstance(self.cell, Layer):
            return self.cell.trainable_weights
        return []

    @property
    def non_trainable_weights(self):
        if isinstance(self.cell, Layer):
            if not self.trainable:
                return self.cell.weights
            return self.cell.non_trainable_weights
        return []

    @property
    def losses(self):
        if isinstance(self.cell, Layer):
            return self.cell.losses
        return []

    def get_losses_for(self, inputs=None):
        if isinstance(self.cell, Layer):
            cell_losses = self.cell.get_losses_for(inputs)
            return cell_losses + super(ATTRNNWrapper, self).get_losses_for(inputs)
        return super(ATTRNNWrapper, self).get_losses_for(inputs)


In [24]:
aa = Input(shape=(3, 2), dtype='float32')
bb = SimpleRNNCell(2)
cc = CellWrapper(bb)
dd = ATTRNNWrapper(LSTM(cc, return_sequences=True, return_state=False))(aa)
ee = Model(inputs=aa, outputs=dd)
ee.compile(optimizer='adam', loss='mae', metrics=['mae'])
ee.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3, 2)              0         
_________________________________________________________________
rnn_3 (RNN)                  (None, 3, 2)              16        
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


#### Training

In [41]:
print('EPOCH_NUM: %s, BATCH_SIZE %s' % (EPOCH_NUM, BATCH_SIZE))

model = ee
fitted = model.fit(train_X, train_Y,
                   epochs=10,     # How many times to run back_propagation
                   batch_size=2,  # How many data to deal with at one epoch
                   validation_split=0.2,
                   verbose=2,       # 1: progress bar, 2: one line per epoch
                   #validation_data=(testX, testY),  # Validation set
                   shuffle=True,
                   callbacks=[history],
                  )

# Save model
model.save('gru_attention_embedding_model.h5')

EPOCH_NUM: 100, BATCH_SIZE 256
Train on 40 samples, validate on 10 samples
Epoch 1/10
 - 0s - loss: 0.4883 - mean_absolute_error: 0.4883 - val_loss: 0.4634 - val_mean_absolute_error: 0.4634
Epoch 2/10
 - 0s - loss: 0.4962 - mean_absolute_error: 0.4962 - val_loss: 0.4515 - val_mean_absolute_error: 0.4515
Epoch 3/10
 - 0s - loss: 0.4707 - mean_absolute_error: 0.4707 - val_loss: 0.4395 - val_mean_absolute_error: 0.4395
Epoch 4/10
 - 0s - loss: 0.4717 - mean_absolute_error: 0.4717 - val_loss: 0.4276 - val_mean_absolute_error: 0.4276
Epoch 5/10
 - 0s - loss: 0.4313 - mean_absolute_error: 0.4313 - val_loss: 0.4182 - val_mean_absolute_error: 0.4182
Epoch 6/10
 - 0s - loss: 0.3984 - mean_absolute_error: 0.3984 - val_loss: 0.4073 - val_mean_absolute_error: 0.4073
Epoch 7/10
 - 0s - loss: 0.4118 - mean_absolute_error: 0.4118 - val_loss: 0.3973 - val_mean_absolute_error: 0.3973
Epoch 8/10
 - 0s - loss: 0.4204 - mean_absolute_error: 0.4204 - val_loss: 0.3904 - val_mean_absolute_error: 0.3904
Epoch

In [42]:
pprint(history.weights[:4])

[[{'begin_input_11': []},
  {'begin_rnn_3': [array([[ 1.06655514,  0.20870832],
       [-0.62020373, -0.92057413]], dtype=float32),
                   array([[-0.87163007,  0.17475815],
       [ 0.16320671,  0.86440241]], dtype=float32),
                   array([-0.09910914,  0.12928225], dtype=float32),
                   array([[ 0.88622832,  0.0257951 ],
       [-0.1858207 , -0.63723952]], dtype=float32),
                   array([ 0.29415649,  1.09380114], dtype=float32)]}],
 [{'end_input_11': []},
  {'end_rnn_3': [array([[ 1.06612265,  0.20919719],
       [-0.62004465, -0.9200052 ]], dtype=float32),
                 array([[-0.87126297,  0.17480175],
       [ 0.16317376,  0.8641423 ]], dtype=float32),
                 array([-0.09923106,  0.12970468], dtype=float32),
                 array([[ 0.88611281,  0.02598154],
       [-0.18512374, -0.63658291]], dtype=float32),
                 array([ 0.29480287,  1.09283495], dtype=float32)]}],
 [{'begin_input_11': []},
  {'begin_rnn_3'

---
# GRU Decoder with Attention (encoder: `return_sequence=True`)

!['lstm_many_to_many_1'](lstm_many_to_many_1.jpg)

!['lstm_attention'](rnn_with_att.jpg)
!['Overview of the Attention mechanism in an Encoder-Decoder setup'](lstm_attention_3.png)
!['detail_lstm_attention'](detail_attentionmodel1.png)

[Attention Structure 1](https://blog.heuritech.com/2016/01/20/attention-mechanism/)  
[Attention Structure 2](http://www.wildml.com/2016/01/attention-and-memory-in-deep-learning-and-nlp/)  
[Attention Structure 3](https://medium.com/datalogue/attention-in-keras-1892773a4f22)  

`keras.layers.Embedding`:  
> `(nb_words, vocab_size) x (vocab_size, embedding_dim) = (nb_words, embedding_dim)`

### MyGRUAttention (Feed-Forward, Not Recurrent)

In [14]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
import numpy as np
import warnings

from keras import backend as K
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.engine import Layer
from keras.engine import InputSpec
from keras.engine.topology import _object_list_uid
from keras.utils.generic_utils import has_arg

# Legacy support.
from keras.legacy.layers import Recurrent
from keras.legacy import interfaces

from keras.layers import RNN, GRUCell, GRU, SimpleRNNCell
#from keras.layers.recurrent import (_generate_dropout_ones,
#                                    _generate_dropout_mask)


def _generate_dropout_ones(inputs, dims):
    # Currently, CTNK can't instantiate `ones` with symbolic shapes.
    # Will update workaround once CTNK supports it.
    if K.backend() == 'cntk':
        ones = K.ones_like(K.reshape(inputs[:, 0], (-1, 1)))
        return K.tile(ones, (1, dims))
    else:
        return K.ones((K.shape(inputs)[0], dims))


def _generate_dropout_mask(ones, rate, training=None, count=1):
    def dropped_inputs():
        return K.dropout(ones, rate)

    if count > 1:
        return [K.in_train_phase(
            dropped_inputs,
            ones,
            training=training) for _ in range(count)]
    return K.in_train_phase(
        dropped_inputs,
        ones,
        training=training)


class GRUAttentionCell(GRUCell):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        #self.state_size = self.units :for RNN, GRU
        #self.state_size = (self.units, self.units) : for LSTM
        #self.attn_length = 79
        #self.attn_size = 79
        #self.attn_vec_size = self.attn_size
        #self.input_size = None
        self.state_size = (self.units, 79*self.units)
        #self.state_size = (self.units, self.units)
        self.full_inputs = None


    def build(self, input_shape):
        print('cell.build Shape:', input_shape)
        #self.timesteps = input_shape[1]
        input_dim = input_shape[-1]
        self.states = [None, None]
        self.kernel = self.add_weight(shape=(input_dim, self.units * 4),
                                      name='kernel',
                                      initializer=self.kernel_initializer,
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)

        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units * 4),
            name='recurrent_kernel',
            initializer=self.recurrent_initializer,
            regularizer=self.recurrent_regularizer,
            constraint=self.recurrent_constraint)

        self.context_kernel = self.add_weight(
            shape=(input_dim, self.units * 3),
            name='context_kernel',
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units * 4,),
                                        name='bias',
                                        initializer=self.bias_initializer,
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.kernel_z = self.kernel[:,               : self.units * 1]
        self.kernel_r = self.kernel[:, self.units * 1: self.units * 2]
        self.kernel_h = self.kernel[:, self.units * 2: self.units * 3]
        self.kernel_c = self.kernel[:, self.units * 3:]

        self.recurrent_kernel_z = self.recurrent_kernel[:,               : self.units * 1]
        self.recurrent_kernel_r = self.recurrent_kernel[:, self.units * 1: self.units * 2]
        self.recurrent_kernel_h = self.recurrent_kernel[:, self.units * 2: self.units * 3]
        self.recurrent_kernel_c = self.recurrent_kernel[:, self.units * 3:]

        self.context_kernel_z = self.context_kernel[:,               : self.units * 1]
        self.context_kernel_r = self.context_kernel[:, self.units * 1: self.units * 2]
        self.context_kernel_h = self.context_kernel[:, self.units * 2:]

        if self.use_bias:
            self.bias_z = self.bias[              : self.units * 1]
            self.bias_r = self.bias[self.units * 1: self.units * 2]
            self.bias_h = self.bias[self.units * 2: self.units * 3]
            self.bias_c = self.bias[self.units * 3:]
        else:
            self.bias_z = None
            self.bias_r = None
            self.bias_h = None
            self.bias_c = None
        self.built = True

        """
        # Parameter Shapes
        kernel : (input_dim, units)
        recurrent_kernel : (units, units)
        bias : (units, )

        # Inherited Parameters
        kernel : z, r, h
        recurrent_kernel : z, r, h
        bias : z, r, h

        # New Parameters
        input_dim
        kernel_c
        recurrent_kernel_c
        bias_c
        """

    def call(self, inputs, states, training=None):
        
        h_tm1 = states[0]  # previous memory
        if self.full_inputs is None:
            self.full_inputs = states[-1]
        full_inputs = self.full_inputs
        timesteps = K.int_shape(full_inputs)[1]
        print('cell.call Input Shape:', K.int_shape(inputs),
              'Full State len:', len(states),
              'State Shape:', K.int_shape(states[0]),
              'Full h Shape:', K.int_shape(states[-1]))

        if 0 < self.dropout < 1 and self._dropout_mask is None:
            self._dropout_mask = _generate_dropout_mask(
                _generate_dropout_ones(inputs, K.shape(inputs)[-1]),
                self.dropout,
                training=training,
                count=4)
        if (0 < self.recurrent_dropout < 1 and
                self._recurrent_dropout_mask is None):
            self._recurrent_dropout_mask = _generate_dropout_mask(
                _generate_dropout_ones(inputs, self.units),
                self.recurrent_dropout,
                training=training,
                count=4)

        # dropout matrices for input units
        dp_mask = self._dropout_mask
        # dropout matrices for recurrent units
        rec_dp_mask = self._recurrent_dropout_mask

        if self.implementation == 1:
            if 0. < self.dropout < 1.:
                inputs_z = inputs * dp_mask[0]
                inputs_r = inputs * dp_mask[1]
                inputs_h = inputs * dp_mask[2]
                inputs_c = inputs * dp_mask[3]
            else:
                inputs_z = inputs
                inputs_r = inputs
                inputs_h = inputs
                inputs_c = inputs
            x_z = K.dot(inputs_z, self.kernel_z)
            x_r = K.dot(inputs_r, self.kernel_r)
            x_h = K.dot(inputs_h, self.kernel_h)
            x_c = K.dot(inputs_c, self.kernel_c)
            if self.use_bias:
                x_z = K.bias_add(x_z, self.bias_z)
                x_r = K.bias_add(x_r, self.bias_r)
                x_h = K.bias_add(x_h, self.bias_h)
                x_c = K.bias_add(x_c, self.bias_c)

            if 0. < self.recurrent_dropout < 1.:
                h_tm1_z = h_tm1 * rec_dp_mask[0]
                h_tm1_r = h_tm1 * rec_dp_mask[1]
                h_tm1_h = h_tm1 * rec_dp_mask[2]
                h_tm1_c = h_tm1 * rec_dp_mask[3]
            else:
                h_tm1_z = h_tm1
                h_tm1_r = h_tm1
                h_tm1_h = h_tm1
                h_tm1_c = h_tm1

            # calculate the context vector
            #context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)

            # Attention Context Part
            h_tm1_c = K.repeat(h_tm1_c, timesteps)
            #h_tm1_c = full_inputs
            print('h:', K.int_shape(full_inputs), 's0:', K.int_shape(h_tm1_c))
            e = self.activation(h_tm1_c + K.dot(full_inputs, self.recurrent_kernel_c))
            a = K.softmax(e)
            print('A:', K.int_shape(a), 'inputs_c:', K.int_shape(inputs_c), 'full input:', K.int_shape(full_inputs))
            c_t = K.sum(a * full_inputs, axis=1, keepdims=False)
            #c_t = K.batch_dot(a, inputs_c, axes=1)

            print('Done c_t:', K.int_shape(c_t))
            print('Done context_kernel:', K.int_shape(self.context_kernel_z))
            print('Done recurrent_kernel:', self.recurrent_kernel_z)
            print('Done h_tm_z * rec:', K.dot(h_tm1_z, self.recurrent_kernel_z))
            
            # GRU Part
            z = self.recurrent_activation(x_z +
                                          K.dot(h_tm1_z,
                                                self.recurrent_kernel_z) +
                                          K.dot(c_t, self.context_kernel_z))
            r = self.recurrent_activation(x_r +
                                          K.dot(h_tm1_r,
                                                self.recurrent_kernel_r) +
                                          K.dot(c_t, self.context_kernel_r))

            hh = self.activation(x_h +
                                 K.dot(r * h_tm1_h,
                                       self.recurrent_kernel_h) +
                                 K.dot(c_t, self.context_kernel_h))

        else:
            """
            if 0. < self.dropout < 1.:
                inputs *= dp_mask[0]
            matrix_x = K.dot(inputs, self.kernel)
            if self.use_bias:
                matrix_x = K.bias_add(matrix_x, self.bias)
            if 0. < self.recurrent_dropout < 1.:
                h_tm1 *= rec_dp_mask[0]
            matrix_inner = K.dot(h_tm1,
                                 self.recurrent_kernel[:, :2 * self.units])

            x_z = matrix_x[:, :self.units]
            x_r = matrix_x[:, self.units: 2 * self.units]
            recurrent_z = matrix_inner[:, :self.units]
            recurrent_r = matrix_inner[:, self.units: 2 * self.units]

            z = self.recurrent_activation(x_z + recurrent_z)
            r = self.recurrent_activation(x_r + recurrent_r)

            x_h = matrix_x[:, 2 * self.units:]
            recurrent_h = K.dot(r * h_tm1,
                                self.recurrent_kernel[:, 2 * self.units:])
            hh = self.activation(x_h + recurrent_h)
            """
            pass

        h = z * h_tm1 + (1 - z) * hh
        if 0 < self.dropout + self.recurrent_dropout:
            if training is None:
                h._uses_learning_phase = True
                
        print(K.int_shape(states[-1]))
        states = [h] + [states[-1]]
        return h, [h, self.full_inputs]


class MyRNNAttention(RNN):

    def get_initial_state(self, inputs):
        # build an all-zero tensor of shape (samples, output_dim)
        initial_state = K.zeros_like(inputs)  # (samples, timesteps, input_dim)
        initial_state = K.sum(initial_state, axis=(1, 2))  # (samples,)
        initial_state = K.expand_dims(initial_state)  # (samples, 1)
        if hasattr(self.cell.state_size, '__len__'):
            state = [K.tile(initial_state, [1, dim])
                     for dim in self.cell.state_size[:-1]]
        else:
            state = [K.tile(initial_state, [1, self.cell.state_size])]

        full_inputs = [inputs]
        return state + full_inputs

### Modeling

In [15]:
_, timestepX, ndimX = train_X.shape
_, timestepY, ndimY = train_Y.shape
#_, ndimY = seq_Y.shape

print('LATENT_DIM: %s' % LATENT_DIM)
print(timestepX, ndimX)

input_ = Input(shape=(timestepX, ndimX), dtype='float32')
enc = LSTM(ndimX, return_sequences=True)(input_)
GRUAttention = MyRNNAttention(GRUAttentionCell(ndimY), return_sequences=True)
dec = GRUAttention(enc, constants=enc)
act = Dense(ndimY, activation='softmax')(dec)
model = Model(inputs=input_, outputs=act)

#parallel_model = multi_gpu_model(model, gpus=GPU_NUM)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

LATENT_DIM: 64
79 80
cell.build Shape: [(None, 80), (None, 79, 80)]


TypeError: can only concatenate tuple (not "int") to tuple

In [454]:

class MinimalRNNCell(Layer):

    def __init__(self, units, **kwargs):
        self.units = units
        #self.state_size = units
        self.state_size = (units, units * 79)
        super(MinimalRNNCell, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='uniform',
            name='recurrent_kernel')
        self.built = True

    def call(self, inputs, states):
        prev_output = states[0]
        h = K.dot(inputs, self.kernel)
        output = h + K.dot(prev_output, self.recurrent_kernel)
        return output, [output]


In [ ]:
Layer.

In [456]:
a = MinimalRNNCell(ndimY)
a.input_shape

AttributeError: The layer has never been called and thus has no defined input shape.

In [453]:
_, timestepX, ndimX = train_X.shape
_, timestepY, ndimY = train_Y.shape
#_, ndimY = seq_Y.shape

print('LATENT_DIM: %s' % LATENT_DIM)
print(timestepX, ndimX)

input_ = Input(shape=(timestepX, ndimX), dtype='float32')
enc = LSTM(ndimX, return_sequences=True)(input_)
#Cel = MinimalRNNCell(ndimY)
dec = RNN(MinimalRNNCell(ndimY), return_sequences=True)(enc, enc)
act = Dense(ndimY, activation='softmax')(dec)
model = Model(inputs=input_, outputs=act)

#parallel_model = multi_gpu_model(model, gpus=GPU_NUM)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

LATENT_DIM: 64
79 80


ValueError: An initial_state was passed that is not compatible with `cell.state_size`. Received `state_spec`=[<keras.engine.topology.InputSpec object at 0x7f70d9075a90>]; However `cell.state_size` is (110, 8690)

In [ ]:
### class CellWrapper(Layer):
    def __init__(self, cell, **kwargs):
        self._cell = cell
        super(CellWrapper, self).__init__(**kwargs)
        #self._cell.__init__(*args, **kwargs)

    @property
    def units(self):
        return self._cell.units

    @property
    def activation(self):
        return self._cell.activation

    @property
    def use_bias(self):
        return self._cell.use_bias

    @property
    def kernel_initializer(self):
        return self._cell.kernel_initializer

    @property
    def recurrent_initializer(self):
        return self._cell.recurrent_initializer

    @property
    def bias_initializer(self):
        return self._cell.bias_initializer

    @property
    def kernel_regularizer(self):
        return self._cell.kernel_regularizer

    @property
    def recurrent_regularizer(self):
        return self._cell.recurrent_regularizer

    @property
    def bias_regularizer(self):
        return self._cell.bias_regularizer

    @property
    def kernel_constraint(self):
        return self._cell.kernel_constraint

    @property
    def recurrent_constraint(self):
        return self._cell.recurrent_constraint

    @property
    def bias_constraint(self):
        return self._cell.bias_constraint

    @property
    def dropout(self):
        return self._cell.dropout

    @property
    def recurrent_dropout(self):
        return self._cell.recurrent_dropout

    @property
    def state_size(self):
        return self._cell.state_size

    @property
    def output_shape(self):
        return self._cell.output_shape

    @property
    def implementation(self):
        return self.cell.implementation

    @property
    def weights(self):
        return [self._cell.weights, self.weights]

    def get_weights(self):
        return [self._cell.get_weights(), self.get_weights()]

    def build(self, input_shape):
        self._cell.build(input_shape)
        
        if self.built is None:
            self.built = True

    def call(self, inputs, states, training=None):

        cell_output, new_state = self._cell.call(inputs, states, training=None)

        output = cell_output
        state = new_state

        return output, state

In [441]:
_, timestepX, ndimX = train_X.shape
_, timestepY, ndimY = train_Y.shape
#_, ndimY = seq_Y.shape

print('LATENT_DIM: %s' % LATENT_DIM)
print(timestepX, ndimX)

input_ = Input(shape=(timestepX, ndimX), dtype='float32')
enc = LSTM(ndimX, return_sequences=True)(input_)
Cel = GRUAttentionCell(ndimX)
dec = RNN(Cel, return_sequences=True)(enc)
act = Dense(ndimY, activation='softmax')(dec)
model = Model(inputs=input_, outputs=act)

#parallel_model = multi_gpu_model(model, gpus=GPU_NUM)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

LATENT_DIM: 64
79 80
cell.build Shape: (None, 80)
cell.call Input Shape: (None, 80) Full State len: 2 State Shape: (None, 80) Full h Shape: (None, 6320)
h: (None, 6320) s0: (None, 6320, 80)


ValueError: Dimensions must be equal, but are 6320 and 80 for 'rnn_26/MatMul_4' (op: 'MatMul') with input shapes: [?,6320], [80,80].

In [409]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import

from keras import activations
from keras import backend as K
from keras.engine import InputSpec
from keras.engine import Layer
from keras.layers import concatenate
from keras.layers import has_arg
from keras import initializers
from keras import regularizers
from keras import constraints


class _RNNAttentionCell(Layer):
    """Base class for recurrent attention mechanisms.

    This base class implements the RNN cell interface and defines a standard
    way for attention mechanisms to interact with a (wrapped) "core" RNN cell
    (such as the `SimpleRNNCell`, `GRUCell` or `LSTMCell`).

    The main idea is that the attention mechanism, implemented by
    `attention_call` in extensions of this class, computes an "attention
    encoding", based on the attended input as well as the input and the core
    cell state(s) at the current time step, which will be used as modified
    input for the core cell.

    # Arguments
        cell: A RNN cell instance. The cell to wrap by the attention mechanism.
            A RNN cell is a class that has:
            - a `call(input_at_t, states_at_t)` method, returning
                `(output_at_t, states_at_t_plus_1)`.
            - a `state_size` attribute. This can be a single integer
                (single state) in which case it is the size of the recurrent
                state (which should be the same as the size of the cell
                output). This can also be a list/tuple of integers (one size
                per state). In this case, the first entry (`state_size[0]`)
                should be the same as the size of the cell output.
        attend_after: Boolean (default False). If True, the attention
            transformation defined by `attention_call` will be applied after
            the core cell transformation (and the attention encoding will be
            used as input for core cell transformation next time step).
        concatenate_input: Boolean (default True). If True the concatenation of
            the attention encoding and the original input will be used as input
            for the core cell transformation. If set to False, only the
            attention encoding will be used as input for the core cell
            transformation.

    # Abstract Methods and Properties
        Extension of this class must implement:
            - `attention_build` (method): Builds the attention transformation
              based on input shapes.
            - `attention_call` (method): Defines the attention transformation
              returning the attention encoding.
            - `attention_size` (property): After `attention_build` has been
              called, this property should return the size (int) of the
              attention encoding. Do this by setting `_attention_size` in scope
              of `attention_build` or by implementing `attention_size`
              property.
        Extension of this class can optionally implement:
            - `attention_state_size` (property): Default [`attention_size`].
              If the attention mechanism has it own internal states (besides
              the attention encoding which is by default the only part of
              `attention_states`) override this property accordingly.
        See docs of the respective method/property for further details.

    # Details of interaction between attention and cell transformations
        Let "cell" denote core (wrapped) RNN cell and "att(cell)" the complete
        attentive RNN cell defined by this class. We write the core cell
        transformation as:

            y{t}, s_cell{t+1} = cell.call(x{t}, s_cell{t})

        where y{t} denotes the output, x{t} the input at and s_cell{t} the core
        cell state(s) at time t and s_cell{t+1} the updated state(s).

        We can then write the complete "attentive" cell transformation as:

            y{t}, s_att(cell){t+1} = att(cell).call(x{t}, s_att(cell){t},
                                                    constants=attended)

        where s_att(cell) denotes the complete states of the attentive cell,
        which consists of the core cell state(s) followed but the attention
        state(s), and attended denotes the tensor attended to (note: no time
        indexing as this is the same constant input at each time step).

        Internally, this is how the attention transformation, implemented by
        `attention_call`, interacts with the core cell transformation
        `cell.call`:

        - with `attend_after=False` (default):
            a{t}, s_att{t+1} = att(cell).attention_call(x_t, s_cell{t},
                                                        attended, s_att{t})
            with `concatenate_input=True` (default):
                x'{t} = [x{t}, a{t}]
            else:
                x'{t} = a{t}
            y{t}, s_cell{t+1} = cell.call(x'{t}, s_cell{t})

        - with `attend_after=True`:
            with `concatenate_input=True` (default):
                x'{t} = [x{t}, a{t-1}]
            else:
                x'{t} = a{t-1}
            y{t}, s_cell{t+1} = cell.call(x'{t}, s_cell{t})
            a{t}, s_att{t+1} = att(cell).attention_call(x_t, s_cell{t+1},
                                                        attended, s_att{t})

        where a{t} denotes the attention encoding, s_att{t} the attention
        state(s), x'{t} the modified core cell input and [x{.}, a{.}] the
        (tensor) concatenation of the input and attention encoding.
    """

    def __init__(self, cell,
                 attend_after=False,
                 concatenate_input=False,
                 **kwargs):
        self.cell = cell  # must be set before calling super
        super(_RNNAttentionCell, self).__init__(**kwargs)
        self.attend_after = attend_after
        self.concatenate_input = concatenate_input
        self.attended_spec = None
        self._attention_size = None

    def attention_call(self,
                       inputs,
                       cell_states,
                       attended,
                       attention_states,
                       training=None):
        """The main logic for computing the attention encoding.

        # Arguments
            inputs: The input at current time step.
            cell_states: States for the core RNN cell.
            attended: The same tensor(s) to attend at each time step.
            attention_states: States dedicated for the attention mechanism.
            training: whether run in training mode or not

        # Returns
            attention_h: The computed attention encoding at current time step.
            attention_states: States to be passed to next `attention_call`. By
                default this should be [`attention_h`].
                NOTE: if additional states are used, these should be appended
                after `attention_h`, i.e. `attention_states[0]` should always
                be `attention_h`.
        """
        raise NotImplementedError(
            '`attention_call` must be implemented by extensions of `{}`'.format(
                self.__class__.__name__))

    def attention_build(self, input_shape, cell_state_size, attended_shape):
        """Build the attention mechanism.

        NOTE: `self._attention_size` should be set in this method to the size
        of the attention encoding (i.e. size of first `attention_states`)
        unless `attention_size` property is implemented in another way.

        # Arguments
            input_shape: Tuple of integers. Shape of the input at a single time
                step.
            cell_state_size: List of tuple of integers.
            attended_shape: List of tuple of integers.

            NOTE: both `cell_state_size` and `attended_shape` will always be
            lists - for simplicity. For example: even if (wrapped)
            `cell.state_size` is an integer, `cell_state_size` will be a list
            of this one element.
        """
        raise NotImplementedError(
            '`attention_build` must be implemented by extensions of `{}`'.format(
                self.__class__.__name__))

    @property
    def attention_size(self):
        """Size off attention encoding, an integer.
        """
        if self._attention_size is None and self.built:
            raise NotImplementedError(
                'extensions of `{}` must either set property `_attention_size`'
                ' in `attention_build` or implement the or implement'
                ' `attention_size` in some other way'.format(
                    self.__class__.__name__))

        return self._attention_size

    @property
    def attention_state_size(self):
        """Size of attention states, defaults to `attention_size`, an integer.

        Modify this property to return list of integers if the attention
        mechanism has several internal states. Note that the first size should
        always be the size of the attention encoding, i.e.:
            `attention_state_size[0]` = `attention_size`
        """
        return self.attention_size

    @property
    def state_size(self):
        """Size of states of the complete attentive cell, a tuple of integers.

        The attentive cell's states consists of the core RNN cell state size(s)
        followed by attention state size(s). NOTE it is important that the core
        cell states are first as the first state of any RNN cell should be same
        as the cell's output.
        """
        state_size_s = []
        for state_size in [self.cell.state_size, self.attention_state_size]:
            if hasattr(state_size, '__len__'):
                state_size_s += list(state_size)
            else:
                state_size_s.append(state_size)

        return tuple(state_size_s)

    def call(self, inputs, states, constants, training=None):
        """Complete attentive cell transformation.
        """
        attended = constants
        cell_states = states[:self._num_wrapped_states]
        attention_states = states[self._num_wrapped_states:]

        if self.attend_after:
            attention_call = self.call_attend_after
        else:
            attention_call = self.call_attend_before

        return attention_call(inputs=inputs,
                              cell_states=cell_states,
                              attended=attended,
                              attention_states=attention_states,
                              training=training)

    def call_attend_before(self,
                           inputs,
                           cell_states,
                           attended,
                           attention_states,
                           training=None):
        """Complete attentive cell transformation, if `attend_after=False`.
        """
        attention_h, new_attention_states = self.attention_call(
            inputs=inputs,
            cell_states=cell_states,
            attended=attended,
            attention_states=attention_states,
            training=training)

        if self.concatenate_input:
            cell_input = concatenate([attention_h, inputs])
        else:
            cell_input = attention_h

        if has_arg(self.cell.call, 'training'):
            output, new_cell_states = self.cell.call(cell_input, cell_states,
                                                     training=training)
        else:
            output, new_cell_states = self.cell.call(cell_input, cell_states)

        return output, new_cell_states + new_attention_states

    def call_attend_after(self,
                          inputs,
                          cell_states,
                          attended,
                          attention_states,
                          training=None):
        """Complete attentive cell transformation, if `attend_after=True`.
        """
        attention_h_previous = attention_states[0]

        if self.concatenate_input:
            cell_input = concatenate([attention_h_previous, inputs])
        else:
            cell_input = attention_h_previous

        if has_arg(self.cell.call, 'training'):
            output, new_cell_states = self.cell.call(cell_input, cell_states,
                                                     training=training)
        else:
            output, new_cell_states = self.cell.call(cell_input, cell_states)

        attention_h, new_attention_states = self.attention_call(
            inputs=inputs,
            cell_states=new_cell_states,
            attended=attended,
            attention_states=attention_states,
            training=training)

        return output, new_cell_states, new_attention_states

    @staticmethod
    def _num_elements(x):
        if hasattr(x, '__len__'):
            return len(x)
        else:
            return 1

    @property
    def _num_wrapped_states(self):
        return self._num_elements(self.cell.state_size)

    @property
    def _num_attention_states(self):
        return self._num_elements(self.attention_state_size)

    def build(self, input_shape):
        """Builds attention mechanism and wrapped cell (if keras layer).

        Arguments:
            input_shape: list of tuples of integers, the input feature shape
                (inputs sequence shape without time dimension) followed by
                constants (i.e. attended) shapes.
        """
        if not isinstance(input_shape, list):
            raise ValueError('input shape should contain shape of both cell '
                             'inputs and constants (attended)')

        attended_shape = input_shape[1:]
        input_shape = input_shape[0]
        self.attended_spec = [InputSpec(shape=shape) for shape in attended_shape]
        if isinstance(self.cell.state_size, int):
            cell_state_size = [self.cell.state_size]
        else:
            cell_state_size = list(self.cell.state_size)
        self.attention_build(
            input_shape=input_shape,
            cell_state_size=cell_state_size,
            attended_shape=attended_shape,
        )

        if isinstance(self.cell, Layer):
            cell_input_shape = (input_shape[0],
                                self.attention_size +
                                input_shape[-1] if self.concatenate_input
                                else self._attention_size)
            self.cell.build(cell_input_shape)

        self.built = True

    def compute_output_shape(self, input_shape):
        if hasattr(self.cell.state_size, '__len__'):
            cell_output_dim = self.cell.state_size[0]
        else:
            cell_output_dim = self.cell.state_size

        return input_shape[0], cell_output_dim

    @property
    def trainable_weights(self):
        return super(_RNNAttentionCell, self).trainable_weights + \
               self.cell.trainable_weights

    @property
    def non_trainable_weights(self):
        return super(_RNNAttentionCell, self).non_trainable_weights + \
               self.cell.non_trainable_weights

    def get_config(self):
        config = {'attend_after': self.attend_after,
                  'concatenate_input': self.concatenate_input}

        cell_config = self.cell.get_config()
        config['cell'] = {'class_name': self.cell.__class__.__name__,
                          'config': cell_config}
        base_config = super(_RNNAttentionCell, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class MixtureOfGaussian1DAttention(_RNNAttentionCell):
    """RNN attention mechanism for attending sequences.

    The attention encoding (passed to the wrapped core RNN cell) is obtained by
    letting the attention mechanism predict a Mixture of Gaussian distribution
    (MoG) over the time dimension of the attended feature sequence. The
    attention encoding is taken as the weighted sum of all features - where the
    weight is given by the probability density function (evaluated in the
    respective time step) according the predicted MoG distribution.

    # Arguments
        components: Positive integer, the number of mixture components to use
            (for each head, see below).
        heads: Positive integer (Default 1), the number of independent "read
            heads" to use. Each head produces an independent (sub) attention
            encoding, by predicting an independent MoG each. The (full)
            attention encoding passed to the wrapped core RNN cell is the
            concatenation of the attention encodings from each head. See "Notes
            on multiple heads vs multiple components" below.
        mu_activation: The activation function applied (after learnt linear
            transformation) for mu:s (expectation value/location) of each
            Gaussian component.
        sigma_activation: The activation function applied (after learnt linear
            transformation) for sigma:s (standard deviation) of each Gaussian
            component. *NOTE* that this function should only return values > 0.
        sigma_epsilon: Positive Float, this value is added to sigma to force it
            to be at least this value.
        predict_delta_mu: Boolean (Default True), whether or not to let the
            attention mechanism to predict the _change_ in location (mu) of
            each mixture component. This is recommended as it usually leads to
            more stable convergence. By passing a `mu_activation` that always
            returns a value > 0 and having `predict_delta_mu=True` it is
            enforced that the attention mechanism "parses" the attended
            sequence "from start to end" as the attention can not be moved
            backwards.
        For initializers, regularizers & constraints: See docs of Dense layer.

    # Notes on multiple heads vs multiple components
        A single head can "attend to multiple parts of the sequence" by
        using multiple components. However, the features from the location of
        the components are averaged together by a weighted sum (no
        information is kept on their internal ordering for example). With
        multiple heads, on the other side, the attention mechanism can "pick
        out" features from multiple locations without averaging them, and
        passing them "intact" to the core RNN cell. This is done at the cost of
        a larger input vector to, and thereby more parameters of, the core RNN
        cell.

    # Example - Machine Translation with Attention and "teacher forcing"
        # NOTE that this is a minimal naive example, this setup will not
        # perform well for machine translation in general.
        # TODO add `examples/machine_translation_with_attention.py`
        # with performing setup

        input_english = Input((None, tokens_english))
        target_french_tm1 = Input((None, tokens_french))

        cell = MixtureOfGaussian1DAttention(LSTMCell(64), components=3, heads=3)
        attention_lstm = RNN(cell, return_sequences=True)
        h_sequence = attention_lstm(target_french_tm1, constants=input_english)
        output_layer = TimeDistributed(Dense(tokens_french, activation='softmax'))
        predicted_french = output_layer(h_sequence)

        train_model = Model(
            inputs=[target_french_tm1, input_english],
            outputs=predicted_french
        )
        model.compile(optimizer='Adam', loss='categorical_crossentropy')
        model.fit(
            x=[french_text[:, :-1], english_text],
            y=french_text[:, 1:],
            epochs=10
        )
    """
    def __init__(self, cell,
                 components,
                 heads=1,
                 mu_activation=None,
                 sigma_activation='exponential',
                 sigma_epsilon=1e-3,
                 predict_delta_mu=True,  # TODO alternative name `cumulative_mu`?
                 kernel_initializer='glorot_uniform',  # FIXME most likely not optimal
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(MixtureOfGaussian1DAttention, self).__init__(cell, **kwargs)
        self.components = components
        self.heads = heads
        self.mu_activation = activations.get(mu_activation)
        self.sigma_activation = activations.get(sigma_activation)
        self.sigma_epsilon = sigma_epsilon
        self.predict_delta_mu = predict_delta_mu
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

    @property
    def attention_state_size(self):
        """Size of states dedicated for the attention mechanism.

        If self.predict_delta_mu is True, mu (the "location") for all heads'
        components needs to be forwarded to next time step and is therefore
        added to the attention states.
        """
        attention_state_size = [self.attention_size]
        if self.predict_delta_mu:
            mu_size = self.components * self.heads
            attention_state_size.append(mu_size)

        return attention_state_size

    def attention_call(self,
                       inputs,
                       cell_states,
                       attended,
                       attention_states,
                       training=None):
        # only one attended sequence for now (verified in build)
        [attended] = attended
        mu_tm1 = attention_states[1] if self.predict_delta_mu else None

        mog_input = concatenate([inputs, cell_states[0]])
        params = K.bias_add(K.dot(mog_input, self.kernel), self.bias)

        # dynamic creation of time index
        # TODO check support by all backends
        # TODO faster with non-dynamic if size of time dimension is fixed?
        time_idx = K.arange(K.shape(attended)[1], dtype='float32')
        time_idx = K.expand_dims(K.expand_dims(time_idx, 0), -1)

        if self.heads == 1:
            attention_h, mu = self._get_attention_h_and_mu(params, attended,
                                                           mu_tm1, time_idx)
        else:
            c = self.components
            attention_h_s, mu_s = zip(*[
                self._get_attention_h_and_mu(
                    params=params[..., c * i * 3:c * (i+1) * 3],
                    attended=attended,
                    mu_tm1=(mu_tm1[..., c * i:c * (i+1)]
                            if self.predict_delta_mu else None),
                    time_idx=time_idx
                ) for i in range(self.heads)
            ])
            attention_h = concatenate(list(attention_h_s))
            mu = concatenate(list(mu_s))

        new_attention_states = [attention_h]
        if self.predict_delta_mu:
            new_attention_states.append(mu)

        return attention_h, new_attention_states

    def _get_attention_h_and_mu(self, params, attended, mu_tm1, time_idx):
        """Computes the attention encoding for "one head".

        # Arguments
            params: The MoG params (before activation) for one head.
            attended: The attended sequence (tensor).
            mu_tm1: mu from previous time step (tensor) if self.use_delta is
                True otherwise None.
            time_idx: Time index of the attended (tensor).

        # Returns
            attention_h: The attention encoding for the attention of one head.
            mu: the location(s) of each mixture component for one head.
        """
        def sigma_activation(x):
            return self.sigma_activation(x) + self.sigma_epsilon

        mixture_weights, mu, sigma = [
            activation(params[..., i * self.components:(i + 1) * self.components])
            for i, activation in enumerate(
                [K.softmax, self.mu_activation, sigma_activation])]

        if self.predict_delta_mu:
            mu += mu_tm1

        mixture_weights_, mu_, sigma_ = [
            K.expand_dims(p, 1) for p in [mixture_weights, mu, sigma]]

        attention_w = K.sum(
            mixture_weights_ * K.exp(- sigma_ * K.square(mu_ - time_idx)),
            # NOTE no normalisation was carried out in original paper by A. Graves
            axis=-1,
            keepdims=True
        )
        attention_h = K.sum(attention_w * attended, axis=1)

        return attention_h, mu

    def attention_build(self, input_shape, cell_state_size, attended_shape):
        if not len(attended_shape) == 1:
            raise ValueError('only a single attended supported')
        attended_shape = attended_shape[0]
        if not len(attended_shape) == 3:
            raise ValueError('only support attending tensors with dim=3')

        # NOTE _attention_size must always be set in `attention_build`
        self._attention_size = attended_shape[-1] * self.heads
        mog_in_dim = (input_shape[-1] + cell_state_size[0])
        mog_out_dim = self.heads * self.components * 3
        self.kernel = self.add_weight(
            shape=(mog_in_dim, mog_out_dim),
            initializer=self.kernel_initializer,
            name='kernel',
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint)
        self.bias = self.add_weight(shape=(mog_out_dim,),
                                    initializer=self.bias_initializer,
                                    name='bias',
                                    regularizer=self.bias_regularizer,
                                    constraint=self.bias_constraint)

    def get_config(self):
        config = {
            'components': self.components,
            'heads': self.heads,
            'mu_activation': activations.serialize(self.mu_activation),
            'sigma_activation': activations.serialize(self.sigma_activation),
            'sigma_epsilon': self.sigma_epsilon,
            'predict_delta_mu': self.predict_delta_mu,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(MixtureOfGaussian1DAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [411]:

class GRUAttentionCell(GRUCell):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        #self.state_size = self.units :for RNN, GRU
        #self.state_size = (self.units, self.units) : for LSTM
        self.attn_length = 79
        self.attn_size = 79
        self.attn_vec_size = self.attn_size
        self.input_size = None
        self.state_size = (self.units, self.units)

    """
    @property
    def state_size(self):
    size = (self._cell.state_size, self._attn_size,
            self._attn_size * self._attn_length)
    if self._state_is_tuple:
      return size
    else:
      return sum(list(size))
    """


    def get_initial_state(self, inputs):
        # build an all-zero tensor of shape (samples, output_dim)
        initial_state = K.zeros_like(inputs)  # (samples, timesteps, input_dim)
        initial_state = K.sum(initial_state, axis=(1, 2))  # (samples,)
        initial_state = K.expand_dims(initial_state)  # (samples, 1)
        if hasattr(self.cell.state_size, '__len__'):
            state = [K.tile(initial_state, [1, dim])
                     for dim in self.cell.state_size[:-1]]
        else:
            state = [K.tile(initial_state, [1, self.cell.state_size])]

        return state


    def build(self, input_shape):
        #self.timesteps = input_shape[0]
        print('build cell:', input_shape)
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units * 4),
                                      name='kernel',
                                      initializer=self.kernel_initializer,
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)

        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units * 4),
            name='recurrent_kernel',
            initializer=self.recurrent_initializer,
            regularizer=self.recurrent_regularizer,
            constraint=self.recurrent_constraint)

        self.context_kernel = self.add_weight(
            shape=(input_dim, self.units * 3),
            name='context_kernel',
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units * 4,),
                                        name='bias',
                                        initializer=self.bias_initializer,
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.kernel_z = self.kernel[:,               : self.units * 1]
        self.kernel_r = self.kernel[:, self.units * 1: self.units * 2]
        self.kernel_h = self.kernel[:, self.units * 2: self.units * 3]
        self.kernel_c = self.kernel[:, self.units * 3:]

        self.recurrent_kernel_z = self.recurrent_kernel[:,               : self.units * 1]
        self.recurrent_kernel_r = self.recurrent_kernel[:, self.units * 1: self.units * 2]
        self.recurrent_kernel_h = self.recurrent_kernel[:, self.units * 2: self.units * 3]
        self.recurrent_kernel_c = self.recurrent_kernel[:, self.units * 3:]

        self.context_kernel_z = self.context_kernel[:,               : self.units * 1]
        self.context_kernel_r = self.context_kernel[:, self.units * 1: self.units * 2]
        self.context_kernel_h = self.context_kernel[:, self.units * 2:]

        if self.use_bias:
            self.bias_z = self.bias[              : self.units * 1]
            self.bias_r = self.bias[self.units * 1: self.units * 2]
            self.bias_h = self.bias[self.units * 2: self.units * 3]
            self.bias_c = self.bias[self.units * 3:]
        else:
            self.bias_z = None
            self.bias_r = None
            self.bias_h = None
            self.bias_c = None
        self.built = True

        """
        # Parameter Shapes
        kernel : (input_dim, units)
        recurrent_kernel : (units, units)
        bias : (units, )

        # Inherited Parameters
        kernel : z, r, h
        recurrent_kernel : z, r, h
        bias : z, r, h

        # New Parameters
        input_dim
        kernel_c
        recurrent_kernel_c
        bias_c
        """

    def call(self, inputs, states, training=None,
             constants=None):
        print('cell.call Input Shape:', K.int_shape(inputs), 'State Shape:', K.int_shape(states[0]))
        h_tm1 = states[0]  # previous memory
        full_inputs = constants
        timesteps = K.int_shape(full_inputs)[1]


        if 0 < self.dropout < 1 and self._dropout_mask is None:
            self._dropout_mask = _generate_dropout_mask(
                _generate_dropout_ones(inputs, K.shape(inputs)[-1]),
                self.dropout,
                training=training,
                count=4)
        if (0 < self.recurrent_dropout < 1 and
                self._recurrent_dropout_mask is None):
            self._recurrent_dropout_mask = _generate_dropout_mask(
                _generate_dropout_ones(inputs, self.units),
                self.recurrent_dropout,
                training=training,
                count=4)

        # dropout matrices for input units
        dp_mask = self._dropout_mask
        # dropout matrices for recurrent units
        rec_dp_mask = self._recurrent_dropout_mask

        if self.implementation == 1:
            if 0. < self.dropout < 1.:
                inputs_z = inputs * dp_mask[0]
                inputs_r = inputs * dp_mask[1]
                inputs_h = inputs * dp_mask[2]
                inputs_c = inputs * dp_mask[3]
            else:
                inputs_z = inputs
                inputs_r = inputs
                inputs_h = inputs
                inputs_c = inputs
            x_z = K.dot(inputs_z, self.kernel_z)
            x_r = K.dot(inputs_r, self.kernel_r)
            x_h = K.dot(inputs_h, self.kernel_h)
            x_c = K.dot(inputs_c, self.kernel_c)
            if self.use_bias:
                x_z = K.bias_add(x_z, self.bias_z)
                x_r = K.bias_add(x_r, self.bias_r)
                x_h = K.bias_add(x_h, self.bias_h)
                x_c = K.bias_add(x_c, self.bias_c)

            if 0. < self.recurrent_dropout < 1.:
                h_tm1_z = h_tm1 * rec_dp_mask[0]
                h_tm1_r = h_tm1 * rec_dp_mask[1]
                h_tm1_h = h_tm1 * rec_dp_mask[2]
                h_tm1_c = h_tm1 * rec_dp_mask[3]
            else:
                h_tm1_z = h_tm1
                h_tm1_r = h_tm1
                h_tm1_h = h_tm1
                h_tm1_c = h_tm1

            # calculate the context vector
            #context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)

            # Attention Context Part
            #h_tm1_c = K.repeat(h_tm1_c, timesteps)
            h_tm1_c = full_inputs
            e = self.activation(x_c + K.dot(h_tm1_c, self.recurrent_kernel_c))
            a = K.softmax(e)
            print('A:', K.int_shape(a), 'inputs_c:', K.int_shape(inputs_c), 'full input:', K.int_shape(full_inputs))
            c_t = K.dot(a, inputs_c)

            # GRU Part
            z = self.recurrent_activation(x_z +
                                          K.dot(h_tm1_z,
                                                self.recurrent_kernel_z) +
                                          K.dot(c_t, self.context_kernel_z))
            r = self.recurrent_activation(x_r +
                                          K.dot(h_tm1_r,
                                                self.recurrent_kernel_r) +
                                          K.dot(c_t, self.context_kernel_r))

            hh = self.activation(x_h +
                                 K.dot(r * h_tm1_h,
                                       self.recurrent_kernel_h) +
                                 K.dot(c_t, self.context_kernel_h))

        else:
            """
            if 0. < self.dropout < 1.:
                inputs *= dp_mask[0]
            matrix_x = K.dot(inputs, self.kernel)
            if self.use_bias:
                matrix_x = K.bias_add(matrix_x, self.bias)
            if 0. < self.recurrent_dropout < 1.:
                h_tm1 *= rec_dp_mask[0]
            matrix_inner = K.dot(h_tm1,
                                 self.recurrent_kernel[:, :2 * self.units])

            x_z = matrix_x[:, :self.units]
            x_r = matrix_x[:, self.units: 2 * self.units]
            recurrent_z = matrix_inner[:, :self.units]
            recurrent_r = matrix_inner[:, self.units: 2 * self.units]

            z = self.recurrent_activation(x_z + recurrent_z)
            r = self.recurrent_activation(x_r + recurrent_r)

            x_h = matrix_x[:, 2 * self.units:]
            recurrent_h = K.dot(r * h_tm1,
                                self.recurrent_kernel[:, 2 * self.units:])
            hh = self.activation(x_h + recurrent_h)
            """
            pass

        h = z * h_tm1 + (1 - z) * hh
        if 0 < self.dropout + self.recurrent_dropout:
            if training is None:
                h._uses_learning_phase = True
        return h, [h]


In [442]:
_, timestepX, ndimX = train_X.shape
_, timestepY, ndimY = train_Y.shape
#_, ndimY = seq_Y.shape

print('LATENT_DIM: %s' % LATENT_DIM)
print(timestepX, ndimX)

input_ = Input(shape=(timestepX, ndimX), dtype='float32')
enc = LSTM(ndimX, return_sequences=True)(input_)
Cel = GRUAttentionCell(ndimX)
dec = RNN(Cel, return_sequences=True)(enc)
act = Dense(ndimY, activation='softmax')(dec)
model = Model(inputs=input_, outputs=act)

#parallel_model = multi_gpu_model(model, gpus=GPU_NUM)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

LATENT_DIM: 64
79 80
cell.build Shape: (None, 80)
cell.call Input Shape: (None, 80) Full State len: 2 State Shape: (None, 80) Full h Shape: (None, 6320)
h: (None, 6320) s0: (None, 6320, 80)


ValueError: Dimensions must be equal, but are 6320 and 80 for 'rnn_27/MatMul_4' (op: 'MatMul') with input shapes: [?,6320], [80,80].

In [443]:
_, timestepX, ndimX = train_X.shape
_, timestepY, ndimY = train_Y.shape
#_, ndimY = seq_Y.shape

print('LATENT_DIM: %s' % LATENT_DIM)

input_ = Input(shape=(timestepX, ndimX), dtype='float32')
enc = LSTM(ndimX, return_sequences=True)(input_)
tim = TimeDistributed(Dense(1))(enc)
#dec = GRUAttention(4, return_sequences=True)(enc)
act = Dense(ndimY, activation='softmax')(tim)
model = Model(inputs=input_, outputs=tim)

#parallel_model = multi_gpu_model(model, gpus=GPU_NUM)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

LATENT_DIM: 64
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_172 (InputLayer)       (None, 79, 80)            0         
_________________________________________________________________
lstm_167 (LSTM)              (None, 79, 80)            51520     
_________________________________________________________________
time_distributed_15 (TimeDis (None, 79, 1)             81        
Total params: 51,601
Trainable params: 51,601
Non-trainable params: 0
_________________________________________________________________


### Training

In [254]:
print('EPOCH_NUM: %s, BATCH_SIZE %s' % (EPOCH_NUM, BATCH_SIZE))

fitted = model.fit(train_X, train_Y,
                   epochs=10,     # How many times to run back_propagation
                   batch_size=2,  # How many data to deal with at one epoch
                   validation_split=0.2,
                   verbose=2,       # 1: progress bar, 2: one line per epoch
                   #validation_data=(testX, testY),  # Validation set
                   shuffle=True,
                   #callbacks=[history],
                  )

# Save model
model.save('gru_attention_embedding_model.h5')

EPOCH_NUM: 100, BATCH_SIZE 256


ValueError: Error when checking target: expected activation_10 to have shape (None, 79, 80) but got array with shape (80000, 79, 110)

### Scoring

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 3))
ax.plot(fitted.history['loss'], label='train')
if 'val_loss' in fitted.history.keys():
    ax.plot(fitted.history['val_loss'], label='validation')
ax.legend()
ax.set_xticks(np.arange(EPOCH_NUM))
fig.show()

In [ ]:
train_loss, train_accuracy = fitted.model.evaluate(train_X, train_Y, verbose=0)
print('Accuracy: %.3f' % (train_accuracy * 100))

train_Y_hat_array = fitted.model.predict(train_X)
train_Y_real = output_decoder(train_Y)
train_Y_hat = output_decoder(train_Y_hat_array)
train_X_real = input_decoder(train_X)

print(train_X_real[:3])
print(train_Y_real[:3])
print(train_Y_hat[:3])

* Train mean of RMSE

### Testing

In [ ]:
test_loss, test_accuracy = fitted.model.evaluate(test_X, test_Y, verbose=0)
print('Accuracy: %.3f' % (test_accuracy * 100))

test_Y_hat_array = fitted.model.predict(test_X)
test_Y_real = output_decoder(test_Y)
test_Y_hat = output_decoder(test_Y_hat_array)
test_X_real = input_decoder(test_X)

print(test_X_real[:3])
print(test_Y_real[:3])
print(test_Y_hat[:3])

Done.